### Sumário Técnico — Previsão do Preço do Bitcoin

##### 1. Setup Inicial
- [1.1 Importação das Bibliotecas](#11-Importacao-das-Bibliotecas)
- [1.2 Inicialização da Sessão Spark](#12-inicializacao-da-sessao-spark) 
- [1.3 Carregamento dos Dados](#13-carregamento-dos-dados)


##### 2. Auditoria e Validação da Série Temporal
- [2.1 Análise Estrutural do DataFrame](#21-analise-estrutural-do-dataframe)
- [2.2 Análise Temporal de Integridade](#22-analise-temporal-de-integridade)

##### 3. Análise Exploratória da Série Temporal (EDA)
- [3.1 Transformação Temporal](#31-transformação-temporal)
- [3.2 Visualizações Temporais](#32-visualizações-temporais)

##### 4. Engenharia de Features Temporais
- [4.1 Retornos e Volatilidade](#41-retornos-e-volatilidade)
- [4.2 Decomposição Estrutural da Série (STL)](#42-decomposicao-estrutural-da-serie-stl)
- [4.3 Análise Espectral (FFT)](#43-análise-espectral-fft)
- [4.4 Detecção de Padrões Cíclicos](#44-detecção-de-padrões-cíclicos)
- [4.5 Estrutura e Persistência Temporal](#45-estrutura-e-persistencia-temporal)

##### 5. Análise de Estacionariedade e Transformações
- [5.1 Transformações da Série](#51-transformações-da-série)
- [5.2 Diagnóstico de Estacionariedade](#52-diagnóstico-de-estacionariedade)

##### 6. Pré-Modelagem e Ajustes Estatísticos
- [6.1 Modelos Lineares de Benchmark (ARIMA)](#61-modelos-lineares-de-benchmark-arima)
- [6.2 Diagnósticos de Resíduos](#62-diagnósticos-de-resíduos)

##### 7. Consolidação das Features Temporais
- [Features STL](#features-stl)
- [Features de Retorno](#features-de-retorno)
- [Features de Volatilidade](#features-de-volatilidade)
- [Features ACF/PACF](#features-acfpacf)
- [Features de Frequência (FFT)](#features-de-frequência-fft)
- [Features de Regime](#features-de-regime)
- [Features Cíclicas](#features-cíclicas)

### 1. Setup Inicial
#### 1.1 Importacao das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import plotly.express as px
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from plotly.subplots import make_subplots
from pyspark.sql.functions import col, unix_timestamp
from statsmodels.tsa.seasonal import STL
from scipy.signal import find_peaks, detrend
from scipy.fft import fft, fftfreq
from statsmodels.tsa.stattools import adfuller
import warnings
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import acf, pacf
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as stats
from statsmodels.tsa.seasonal import STL
from src.visualizations.plot_btc import plot_btc_price_timeseries,plot_transformed_series, plot_seasonal_weekly_line, plot_rolling_diagnostics_overlay, plot_btc_boxplot_by_week_comparison, plot_btc_boxplot_by_week, plot_btc_boxplot_by_month_comparison, plot_bitcoin_seasonal_patterns, plot_intraday_price_by_hour,  plot_weekly_seasonality_all_years, plot_seasonal_daily_line, plot_stl_decomposition, plot_fft_spectrum, plot_btc_boxplot_by_dayofyear, plot_histogram_variacao_btc, plot_series_comparativa, plot_acf_diferenciada,plot_rolling_mean_std, plot_btc_boxplot_by_month, plot_acf_pacf, plot_arima_layers, plot_acf_residuos, plot_volatility_rolling,plot_residuos_analysis, plot_btc_candlestick_ohlc, plot_acf_pacf_returns, plot_btc_boxplot_by_hour, plot_log_return_analysis
from src.features.stl_features import extract_stl_features
from src.features.arima_features import extract_arima_features
from src.features.regime_features import extract_regime_features
from src.features.fft_features import extract_fft_features, extract_peak_features, extract_cycle_features, reconstruct_fft, extract_cyclic_features
from scipy.stats import normaltest
from statsmodels.tsa.seasonal import STL
import pandas as pd
from scipy.signal import find_peaks
import numpy as np
from hurst import compute_Hc
from scipy.stats import median_abs_deviation
import numpy as np
from hurst import compute_Hc
import numpy as np
import pandas as pd
from statsmodels.stats.diagnostic import het_arch
import numpy as np
import pandas as pd
from scipy.stats import entropy


#### 1.2 Inicializacao da Sessao Spark

In [ ]:
# INICIAR SESSÃO SPARK COM OTIMIZAÇÕES PARA O MAC M2 (8GB RAM)
spark = SparkSession.builder \
    .appName("Bitcoin_Forecasting") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "8") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "5g") \
    .config("spark.memory.fraction", "0.85") \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.cleaner.referenceTracking.cleanCheckpoints", "false") \
    .config("spark.executor.heartbeatInterval", "60000ms") \
    .config("spark.task.cpus", "2") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .master("local[*]") \
    .getOrCreate()

# REDUZIR LOGS PARA EVITAR POLUIÇÃO NO CONSOLE
spark.sparkContext.setLogLevel("ERROR")

print("\n SparkSession configurada com sucesso!")

#### 1.3 Carregamento dos Dados

In [ ]:
PASTA_FEATURES = "/Users/rodrigocampos/Library/Mobile Documents/com~apple~CloudDocs/bitcoin_features/features_temp/df_bitcoin_features.parquet"
df_features_temp = spark.read.parquet(PASTA_FEATURES)

### 2. Auditoria e Validação da Série Temporal

#### 2.1 Análise Estrutural do DataFrame


##### 🔶 ₿ -----> Tipos de Dados

In [ ]:
df_features_temp.printSchema(  
)

##### 🔶 ₿ -----> Contagem de Linhas e Colunas

In [ ]:
num_linhas = df_features_temp.count()  
# --> Counts the total number of rows in the DataFrame / Conta o número total de linhas no DataFrame <--

num_colunas = len(df_features_temp.columns)  
# --> Counts the total number of columns in the DataFrame / Conta o número total de colunas no DataFrame <--

print(f"df_features_temp possui {num_linhas} linhas e {num_colunas} colunas.")  
# --> Prints the shape of the DataFrame / Imprime a forma (dimensão) do DataFrame <--

##### 🔶 ₿ -----> Verificação de Valores Ausentes

In [ ]:
nulos = df_features_temp.filter(df_features_temp["btc_price_usd"].isNull())
# --> Filters rows where 'btc_price_usd' is null / Filtra as linhas onde 'btc_price_usd' está nulo <--

nulos.select("block_height", "block_timestamp").show(truncate=False)
# --> Shows block height and timestamp for null-price rows / Mostra a altura do bloco e o timestamp para os preços nulos <--

print("Total de blocos com preço nulo:", nulos.count())
# --> Counts how many rows have missing prices / Conta quantas linhas têm preços ausentes <--

##### 🔶 ₿ -----> Vizualização do Dataframe

#### 2.2 Analise Temporal de Integridade

##### 🔶 ₿ -----> Primeira e Última Data (block_timestamp)

In [ ]:
# Calcular o timestamp mínimo e máximo
df_features_temp.select(
    F.min("block_timestamp").alias("Min block_timestamp"),
    F.max("block_timestamp").alias("Max block_timestamp")
).show(truncate=False)
# --> Selects and displays the minimum and maximum timestamps in the 'block_timestamp' column / 
# --> Seleciona e exibe os timestamps mínimo e máximo da coluna 'block_timestamp' <--

##### 🔶 ₿ -----> Primeiro e Último Bloco (block_number)

In [ ]:
primeiro_bloco = df_features_temp.select("block_height").orderBy("block_height").first()[0]
# --> Gets the smallest block height (first block) from the DataFrame / 
# --> Obtém a menor altura de bloco (primeiro bloco) do DataFrame <--

ultimo_bloco = df_features_temp.select("block_height").orderBy(F.desc("block_height")).first()[0]
# --> Gets the largest block height (last block) from the DataFrame / 
# --> Obtém a maior altura de bloco (último bloco) do DataFrame <--

print(f"Primeiro bloco salvo: {primeiro_bloco}")
print(f"Último bloco salvo: {ultimo_bloco}")
# --> Prints the range of saved blocks / Imprime o intervalo de blocos salvos <--

print(df_features_temp.select("block_height").distinct().count())
# --> Counts the number of distinct block heights in the DataFrame / 
# --> Conta o número de alturas de blocos distintas no DataFrame <--

##### 🔶 ₿ -----> Gaps de Blocos

In [ ]:
# Comparar cada bloco com o anterior

windowSpec = Window.orderBy("block_height")
# --> Define a window sorted by block height / Define uma janela ordenada pela altura dos blocos <--

df_blocks = df_features_temp.withColumn("prev_block_height", F.lag("block_height").over(windowSpec))
# --> Creates a column with the previous block height / Cria uma coluna com a altura do bloco anterior <--

df_blocks_filtered = df_blocks.select("block_height", "prev_block_height")
# --> Select only relevant columns before checking for gaps / Seleciona apenas as colunas relevantes antes de verificar gaps <--

df_gaps = df_blocks_filtered.withColumn("gap_detected", (col("block_height") - col("prev_block_height")) > 1)
# --> Creates a boolean column that marks if a gap exists between blocks / Cria uma coluna booleana que marca se há gap entre os blocos <--

df_gaps_filtered = df_gaps.filter(col("gap_detected") == True)
# --> Filters only the rows where a gap was detected / Filtra apenas as linhas onde um gap foi detectado <--

df_gaps_filtered.select("prev_block_height", "block_height").show()
# --> Displays the previous and current block height where a gap occurred / Mostra os blocos com gaps detectados <--

##### 🔶 ₿ -----> Gaps de Tempo

In [ ]:
# Janela ordenada por block_height
windowSpec = Window.orderBy("block_height")
# --> Define a window ordered by block height / Define uma janela ordenada pela altura dos blocos <--

df_with_prev_ts = df_features_temp.withColumn(
    "prev_timestamp", F.lag("block_timestamp").over(windowSpec)
)
# --> Creates a new column with the previous block's timestamp / Cria uma nova coluna com o timestamp do bloco anterior <--

df_with_diff_days = df_with_prev_ts.withColumn(
    "gap_days",
    (F.unix_timestamp("block_timestamp") - F.unix_timestamp("prev_timestamp")) / (60 * 60 * 24)
)
# --> Calculates the time difference in days between consecutive blocks / Calcula a diferença em dias entre blocos consecutivos <--

df_with_day_gaps = df_with_diff_days.withColumn(
    "gap_detected_days", col("gap_days") > 1
)
# --> Creates a boolean column to flag gaps greater than 1 day / Cria uma coluna booleana para marcar gaps maiores que 1 dia <--

df_day_gaps = df_with_day_gaps.filter(col("gap_detected_days") == True)
# --> Filters only rows with time gaps larger than 1 day / Filtra apenas as linhas com gaps de tempo maiores que 1 dia <--

df_day_gaps.select("prev_timestamp", "block_timestamp", "gap_days", "block_height").show(truncate=False)
# --> Displays previous and current timestamps, gap in days, and block height / Exibe timestamps anterior e atual, gap em dias e altura do bloco <--

### 3. Análise Exploratória da Série Temporal (EDA)

#### 3.1 Transformacao Temporal

##### 🔶 ₿ -----> Conversão de Timestamps

In [ ]:
df_time = df_features_temp.toPandas()
# --> Converte o DataFrame PySpark para Pandas / Converts PySpark DataFrame to Pandas <--

df_time["block_timestamp"] = pd.to_datetime(df_time["block_timestamp"])
# --> Garante que o campo de tempo seja do tipo datetime / Ensures timestamp field is datetime type <--

##### 🔶 ₿ -----> Definição de Índice Temporal

In [ ]:
df_time_index = df_time.sort_values("block_timestamp")
# --> Ordena o DataFrame pelo tempo / Sorts the DataFrame by timestamp <--

df_time_index.set_index('block_timestamp', inplace=True)
# --> Define o índice como timestamp (necessário para o resample) / Sets timestamp as index (needed for resample) <--

print(df_time_index.index)
# --> Mostra o novo índice temporal / Displays new datetime index <--

In [ ]:
df_time_index.index.to_series().diff().value_counts()
# --> Calcula a diferença entre índices consecutivos e conta a frequência de cada intervalo /
# --> Computes the difference between consecutive index entries and counts frequency of each interval <--

##### 🔶 ₿ -----> Resampling

In [ ]:
df_time_index = df_time_index.resample('h').mean()
# --> Reamostra os dados com frequência horária, calculando a média por hora / Resamples data to hourly frequency, taking the mean per hour <--

### 3.2 Visualizações Temporais

##### 🔶 ₿ -----> Série Temporal do Preço (USD)

In [ ]:
image_path = "/Users/rodrigocampos/Documents/Bitcoin/project/src/visualizations/BTC_black.png"

In [ ]:
plot_btc_price_timeseries(df_time_index, image_path, resample='1h')

##### 🔶 ₿ -----> Gráfico de Abertura vs. Fechamento

In [ ]:
plot_btc_candlestick_ohlc(df_time_index, image_path, resample='h')

##### 🔶 ₿ -----> Boxplot por Janela Temporal

In [ ]:
plot_btc_boxplot_by_hour(df_time_index, image_path)
plot_btc_boxplot_by_dayofyear(df_time_index, image_path)
plot_btc_boxplot_by_week(df_time_index, image_path)
plot_btc_boxplot_by_month(df_time_index, image_path)

##### 🔶 ₿ -----> Distribuição e Histograma da Variação Percentual

In [ ]:
plot_histogram_variacao_btc(df_time_index, image_path)

### 4. Engenharia de Features Temporais

#### 4.1 Retornos e Volatilidade

---

Retornos e Volatilidade

Retornos Logarítmicos

Dado o preço de um ativo $ P_t $ no tempo $ t $, o **retorno logarítmico** é definido como:

$
r_t = \ln\left(\frac{P_t}{P_{t-1}}\right)
$

Onde:

- $ r_t $: retorno no instante $ t $
- $ \ln $: logaritmo natural
- $ P_t $: preço do ativo no tempo $ t $
- $ P_{t-1} $: preço do ativo no tempo anterior

Este tipo de retorno é amplamente utilizado por possuir propriedades aditivas no tempo, o que facilita a modelagem estatística.

---

Volatilidade Histórica

A **volatilidade histórica** representa a dispersão dos retornos e é calculada como o desvio padrão $ \sigma $ dos retornos $ r_t $:

$
\sigma = \sqrt{\frac{1}{N - 1} \sum_{t=1}^{N} (r_t - \bar{r})^2}
$

Onde:

- $ sigma $: volatilidade (desvio padrão dos retornos)
- $ N $: número de observações
- $ \bar{r} $: média dos retornos

---

Interpretação:

- **Retornos positivos** indicam valorização do ativo.
- **Volatilidade alta** sugere maior risco (grandes variações nos preços).
- Pode-se utilizar janelas móveis para calcular **volatilidade dinâmica** ao longo do tempo.

---

>Observações adicionais:
- A volatilidade anualizada pode ser obtida multiplicando a volatilidade diária por $ \sqrt{252} $, assumindo 252 pregões por ano:
  $
  \sigma_{\text{anual}} = \sigma_{\text{diária}} \cdot \sqrt{252}
$

---

In [ ]:
df_return = df_features_temp.toPandas()
# --> Converte o DataFrame PySpark para Pandas / Converts PySpark DataFrame to Pandas <--

df_return["block_timestamp"] = pd.to_datetime(df_return["block_timestamp"])
# --> Garante que o campo de tempo seja do tipo datetime / Ensures timestamp field is datetime type <--

df_return = df_return.sort_values("block_timestamp")
# --> Ordena o DataFrame pelo tempo / Sorts the DataFrame by timestamp <--

df_return.set_index('block_timestamp', inplace=True)
# --> Define o índice como timestamp (necessário para o resample) / Sets timestamp as index (needed for resample) <--

df_return = df_return.resample('d').mean()
# --> Reamostra os dados com frequência horária, calculando a média por hora / Resamples data to hourly frequency, taking the mean per hour <--

print(df_return.index)
# --> Mostra o novo índice temporal / Displays new datetime index <--

In [ ]:
print(df_return.columns)

In [ ]:

# ================================================================
# EXTRAÇÃO DE RETORNOS E TARGETS / RETURN & TARGET FEATURE ENGINEERING
# ================================================================
def extract_return_features(
    price_series: pd.Series,
    block_height: pd.Series,
    block_timestamp: pd.Series
) -> pd.DataFrame:
    """
    Extrai o retorno logarítmico e targets para modelagem preditiva.
    / Extracts log returns and targets for predictive modeling.

    Parâmetros / Parameters:
    - price_series: pd.Series
        Série temporal de preços / Time series of prices.
    - block_height: pd.Series
        Altura do bloco para manter rastreabilidade / Block height for traceability.
    - block_timestamp: pd.Series
        Carimbo de tempo do bloco / Block timestamp.

    Retorna / Returns:
    - pd.DataFrame com colunas: block_height, block_timestamp, log_return, target_direction, target_return
      / DataFrame with features: block_height, timestamp, log return, binary and continuous targets.
    """

    # ================================
    # BASE TEMPORAL / BASE SETUP
    # ================================
    df = pd.DataFrame({
        "price": price_series.values,
        "block_height": block_height.values,
        "block_timestamp": block_timestamp.values
    })
    # --> Cria DataFrame base com os metadados essenciais / Creates base DataFrame with essential metadata <--

    # ================================
    # ENGENHARIA DE RETORNOS / RETURN FEATURES
    # ================================
    df["log_return"] = np.log(df["price"] / df["price"].shift(1))
    # --> Calcula retorno logarítmico entre períodos / Calculates log return between periods <--

    df["target_direction"] = (df["log_return"].shift(-1) > 0).astype(int)
    # --> Define 1 se o próximo retorno for positivo, 0 caso contrário / Binary target: 1 if next return is positive <--

    df["target_return"] = df["log_return"].shift(-1)
    # --> Define o retorno futuro como variável contínua / Continuous target: next log return <--

    # ================================
    # LIMPEZA FINAL / CLEANUP
    # ================================
    df = df.dropna(subset=["log_return", "target_direction", "target_return"])
    # --> Remove valores nulos gerados pelos shifts / Drop NaNs caused by shifting <--

    # ================================
    # SAÍDA FINAL / FINAL OUTPUT
    # ================================
    return df[["block_height", "block_timestamp", "log_return", "target_direction", "target_return"]]
    # --> Retorna DataFrame final com colunas relevantes / Returns final DataFrame with relevant columns <--

In [ ]:
df_return = df_return.reset_index()

df_returns = extract_return_features(
    price_series=df_return["btc_price_usd"],
    block_height=df_return["block_height"],
    block_timestamp=df_return["block_timestamp"]
)

# Visualizando o resultado
print("DataFrame de Retornos e Targets:")
display(df_returns.head())

In [ ]:
# --> Estatísticas descritivas dos retornos / Descriptive statistics of returns <--
print(df_returns["log_return"].describe())

# --> Assimetria (skewness) da distribuição / Skewness of the distribution <--
print(f"\nSkew: {df_returns['log_return'].skew()}")

# --> Curtose (kurtosis) da distribuição / Kurtosis of the distribution <--
print(f"\nKurt: {df_returns['log_return'].kurt()}")

In [ ]:
plot_log_return_analysis(df_returns["log_return"])

In [ ]:
plot_acf_pacf_returns(df_returns["log_return"], 24)

In [ ]:
plot_volatility_rolling(df_returns, window=30)

In [ ]:

stat, p = normaltest(df_returns["log_return"].dropna())
print(f"p-valor do teste de normalidade: {p}")

In [ ]:


# ================================================================
# EXTRAÇÃO DE FEATURES DE VOLATILIDADE / VOLATILITY FEATURE EXTRACTION
# ================================================================
def extract_volatility_features(
    df: pd.DataFrame,
    return_col: str = "log_return",
    block_height_col: str = "block_height",
    block_timestamp_col: str = "block_timestamp",
    window: int = 24
) -> pd.DataFrame:
    """
    Extrai features de volatilidade com base na série de retornos.
    / Extracts volatility features based on the return series.

    Parâmetros / Parameters:
    - df: pd.DataFrame
        DataFrame contendo a coluna de retornos e metadados / DataFrame with return column and metadata.
    - return_col: str
        Nome da coluna de retornos logarítmicos / Name of log return column.
    - block_height_col: str
        Nome da coluna de altura do bloco / Name of block height column.
    - block_timestamp_col: str
        Nome da coluna de timestamp / Name of timestamp column.
    - window: int
        Tamanho da janela para cálculo da rolling volatility / Rolling window size.

    Retorna / Returns:
    - pd.DataFrame com colunas: block_height, block_timestamp, volatility_rolling, volatility_zscore, volatility_jump_flag
    / DataFrame with volatility features and metadata.
    """

    # ================================
    # PREPARAÇÃO DA BASE
    # ================================
    df = df[[return_col, block_height_col, block_timestamp_col]].copy()
    # --> Seleciona apenas as colunas necessárias / Keep only relevant columns <--

    # ================================
    # CÁLCULO DAS FEATURES DE VOLATILIDADE
    # ================================
    df["volatility_rolling"] = df[return_col].rolling(window).std()
    # --> Desvio padrão móvel como proxy de volatilidade / Rolling standard deviation <--

    df["volatility_zscore"] = (
        df["volatility_rolling"] - df["volatility_rolling"].rolling(window).mean()
    ) / df["volatility_rolling"].rolling(window).std()
    # --> Z-score da volatilidade em relação à média histórica local / Local z-score of volatility <--

    df["volatility_jump_flag"] = (df["volatility_zscore"].abs() > 2).astype(int)
    # --> Flag binária para detectar picos de volatilidade (|z| > 2) / Binary flag for extreme volatility <--

    # ================================
    # RETORNO FINAL
    # ================================
    return df[[block_height_col, block_timestamp_col, "volatility_rolling", "volatility_zscore", "volatility_jump_flag"]]
    # --> Retorna apenas colunas úteis para junção / Return only relevant feature columns <--

In [ ]:
df_volatility = extract_volatility_features(df_returns)
display(df_volatility.tail())

#### 4.2 Decomposicao Estrutural da Serie (STL)

---

Modelo Aditivo de Séries Temporais

A decomposição STL (Seasonal-Trend decomposition using Loess) separa uma série temporal $ Y_t $ em três componentes principais:

$
Y_t = T_t + S_t + R_t
$

Onde:

- $ T_t $: tendência (trend) — representa a variação de longo prazo  
- $ S_t $: sazonalidade (seasonal) — padrões que se repetem em ciclos regulares  
- $ R_t $: resíduo (residual) — ruído ou componente aleatório

---

STL – *Seasonal and Trend decomposition using Loess*

STL é uma técnica robusta que utiliza regressão local (LOESS) para suavizar e estimar cada componente separadamente. Seu diferencial:

- Permite **sazonalidade variável no tempo**
- Suporta **dados com outliers**
- É parametrizável com escolha do período e robustez

---

Interpretação prática:

- A **tendência** $ T_t $ permite identificar a direção geral do fenômeno (ex: crescimento do preço).
- A **sazonalidade** $ S_t $ revela padrões periódicos (ex: ciclos mensais ou semanais).
- O **resíduo** $ R_t $ pode ser analisado para investigar anomalias, choques ou ruído puro.

---

Aplicação:

Essa decomposição é útil para:

- **Pré-processamento** de séries para modelos preditivos
- **Detecção de anomalias** (com base nos resíduos)
- **Análise exploratória** de comportamento estrutural da série

---

Reversibilidade do modelo:

Como se trata de uma decomposição aditiva, podemos sempre reconstruir a série original somando os componentes:

$
Y_t = T_t + S_t + R_t
$

---

##### 🔶 ₿ -----> Tendência, Sazonalidade e Resíduo

In [ ]:
df_stl = df_time_index.resample('h').mean()
# --> Reamostra os dados com frequência horária, calculando a média por hora / Resamples data to hourly frequency, taking the mean per hour <--

df_stl["btc_price_usd"] = df_stl["btc_price_usd"].interpolate()
# --> Interpola valores ausentes no preço do Bitcoin / Interpolates missing Bitcoin price values <--

df_stl = df_stl[df_stl["btc_price_usd"].notna()]
# --> Remove qualquer linha restante com valores ausentes / Drops any remaining rows with missing values <--

In [ ]:
# ================================================================
# APLICAÇÃO DA DECOMPOSIÇÃO STL / STL DECOMPOSITION APPLICATION
# ================================================================
def apply_stl_decomposition(
    df: pd.DataFrame,
    target_col: str,
    block_height_col: str = "block_height",
    block_timestamp_col: str = "block_timestamp",
    period: int = 48
) -> pd.DataFrame:
    """
    Aplica a decomposição STL em uma coluna de série temporal e retorna os componentes.
    / Applies STL decomposition to a time series column and returns its components.

    Parâmetros / Parameters:
    - df: pd.DataFrame
        DataFrame com colunas de preço, timestamp e altura do bloco /
        DataFrame containing price, timestamp and block height.
    - target_col: str
        Nome da coluna alvo para decomposição / Name of the column to decompose.
    - block_height_col: str
        Nome da coluna de altura do bloco / Name of the block height column.
    - block_timestamp_col: str
        Nome da coluna de timestamp / Name of the timestamp column.
    - period: int
        Periodicidade da série para sazonalidade / Seasonality period.

    Retorna / Returns:
    - pd.DataFrame com colunas: block_height, block_timestamp, trend, seasonal, resid /
      DataFrame with STL components and metadata.
    """
    
    # ================================
    # PREPARAÇÃO DA SÉRIE
    # ================================
    df = df[[target_col, block_height_col, block_timestamp_col]].dropna().copy()
    # --> Seleciona e limpa colunas necessárias / Select and clean necessary columns <--

    stl = STL(df[target_col], period=period, robust=True)
    # --> Configura a decomposição STL / Configures STL decomposition <--

    result = stl.fit()
    # --> Executa o ajuste da decomposição / Fits the STL decomposition <--

    df_stl = pd.DataFrame({
        block_height_col: df[block_height_col].values,
        block_timestamp_col: df[block_timestamp_col].values,
        "trend": result.trend,
        "seasonal": result.seasonal,
        "resid": result.resid
    }, index=df.index)
    # --> Cria DataFrame com componentes STL e metadados / Creates DataFrame with STL components <--

    return df_stl
    # --> Retorna apenas colunas úteis para merge / Returns aligned components with metadata <--

In [ ]:
# Série de preço limpa e com índice reiniciado
btc_price_clean = df_stl["btc_price_usd"].dropna().reset_index(drop=True)

# Aplica a decomposição STL com essa série
from statsmodels.tsa.seasonal import STL
result = STL(btc_price_clean, period=48, robust=True).fit()

# Monta o DataFrame decomposto com mesmo índice
df_stl_dec = pd.DataFrame({
    "trend": result.trend,
    "seasonal": result.seasonal,
    "resid": result.resid
}, index=btc_price_clean.index)

##### 🔶 ₿ -----> Tendência, Sazonalidade e Resíduo - Análise Gráfica

In [ ]:
plot_stl_decomposition(
    df_stl=df_stl_dec,
    price_series=df_stl["btc_price_usd"]
)

##### 🔶 ₿ -----> Extração de Features Estruturais (spikiness, curvature, linearity, etc.)

In [ ]:
df_stl = df_stl.reset_index()

# Define a janela da série (por exemplo, 2 dias)
window = df_stl["btc_price_usd"].iloc[-96:]  # 48 * 2 períodos de 30 minutos

features_stl = extract_stl_features(
    series=window,
    block_height=df_stl["block_height"].iloc[-1],
    block_timestamp=df_stl["block_timestamp"].iloc[-1],
    period=5
)

display(features_stl)

##### 🔶 ₿ -----> Análise dos Componentes em Subplots

In [ ]:
# Para df_seasonal
df_seasonal = df_features_temp.toPandas()  # Se precisar de ordenação aqui, pode usar:
df_seasonal = df_seasonal.sort_values("block_timestamp")  # Ordena pelo timestamp

In [ ]:
plot_seasonal_daily_line(df_seasonal, timestamp_col="block_timestamp", price_col="btc_price_usd", ano_alvo=2024)

In [ ]:
plot_seasonal_weekly_line(df_seasonal, timestamp_col="block_timestamp", price_col="btc_price_usd", ano_alvo=2025, image_path=image_path)

In [ ]:
plot_weekly_seasonality_all_years(df_seasonal)

In [ ]:
plot_intraday_price_by_hour(
    df=df_seasonal,
    year_filter=2025,
    image_path="/Users/rodrigocampos/Documents/Bitcoin/project/src/visualizations/BTC_black.png"
)

In [ ]:
# Exemplo:
plot_bitcoin_seasonal_patterns(df_seasonal)

##### 🔶 ₿ -----> Teste Estatístico de Interação via ANOVA

---


ANOVA de Duas Vias com Interação

A ANOVA (Análise de Variância) testa se **as médias de um grupo são estatisticamente diferentes** entre si. No caso de **duas variáveis categóricas**, podemos incluir um **termo de interação** para avaliar se o efeito de uma variável depende da outra.

---

Modelo de regressão com interação:

Seja a variável resposta $ Y $ (neste caso, o preço do Bitcoin) e duas variáveis categóricas: mês ($ M $) e ano ($ A $). O modelo com interação é dado por:

$
Y = \mu + \alpha_M + \beta_A + (\alpha\beta)_{M,A} + \epsilon
$

Onde:

- $ \mu $: média geral
- $ \alpha_M $: efeito do mês
- $ \beta_A $: efeito do ano
- $ (\alpha\beta)_{M,A} $: efeito de interação entre mês e ano
- $ \epsilon $: erro aleatório

---


In [ ]:
# Criar colunas necessárias
df_seasonal["block_timestamp"] = pd.to_datetime(df_seasonal["block_timestamp"])
df_seasonal["Ano"] = df_seasonal["block_timestamp"].dt.year.astype(str)  # precisa ser string para categórico
df_seasonal["Mes"] = df_seasonal["block_timestamp"].dt.month_name()

# ==========================
# Modelo com Interação
# ==========================
modelo_interacao = smf.ols("btc_price_usd ~ C(Mes) * C(Ano)", data=df_seasonal).fit()

# ==========================
# ANOVA para testar significância da interação
# ==========================
from statsmodels.stats.anova import anova_lm
anova_resultado = anova_lm(modelo_interacao)

# Visualizar resultado
print(anova_resultado)

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# ================================================================
# EXTRAÇÃO DE FEATURES SAZONAIS A PARTIR DE MÊS E ANO
# EXTRACTION OF SEASONAL FEATURES BASED ON MONTH AND YEAR
# ================================================================
def extract_seasonal_anova_features(
    df: pd.DataFrame,
    date_col: str = "block_timestamp",
    target_col: str = "btc_price_usd",
    block_height: int = None,
    block_timestamp: pd.Timestamp = None
) -> pd.DataFrame:
    """
    Gera variáveis categóricas de Mês e Ano e realiza ANOVA com interação para identificar sazonalidade.
    / Generates categorical Month-Year variables and performs ANOVA to detect seasonality interactions.
    
    Parâmetros / Parameters:
    - df: pd.DataFrame
        DataFrame contendo a série temporal / DataFrame containing time series.
    - date_col: str
        Nome da coluna de data / Name of timestamp column.
    - target_col: str
        Coluna de valores numéricos alvo / Column with target values.
    - block_height: int
        Altura do bloco de referência / Reference block height.
    - block_timestamp: pd.Timestamp
        Timestamp de referência do bloco / Reference block timestamp.

    Retorna / Returns:
    - pd.DataFrame com colunas: block_height, block_timestamp, anova_p_mes, anova_p_ano, anova_p_interacao /
      DataFrame with tracking columns and ANOVA p-values for month, year, and interaction.
    """
    df = df.copy()

    # ================================
    # EXTRAÇÃO DE MÊS E ANO / EXTRACT MONTH & YEAR
    # ================================
    df[date_col] = pd.to_datetime(df[date_col])
    # --> Garante tipo datetime / Ensures datetime format <--

    df["Ano"] = df[date_col].dt.year.astype(str)
    df["Mes"] = df[date_col].dt.month_name()
    # --> Extrai componentes temporais para ANOVA / Extracts month and year for ANOVA <--

    # ================================
    # AJUSTE DO MODELO COM INTERAÇÃO / FIT INTERACTION MODEL
    # ================================
    formula = f"{target_col} ~ C(Mes) * C(Ano)"
    modelo = smf.ols(formula=formula, data=df).fit()
    # --> Ajusta modelo com interação mês x ano / Fit model with month-year interaction <--

    # ================================
    # ANOVA E P-VALUES / ANOVA AND P-VALUES
    # ================================
    anova_result = anova_lm(modelo)

    p_mes = anova_result.loc["C(Mes)", "PR(>F)"] if "C(Mes)" in anova_result.index else None
    p_ano = anova_result.loc["C(Ano)", "PR(>F)"] if "C(Ano)" in anova_result.index else None
    p_inter = anova_result.loc["C(Mes):C(Ano)", "PR(>F)"] if "C(Mes):C(Ano)" in anova_result.index else None
    # --> P-valores de cada fator e da interação / P-values for month, year and interaction <--

    # ================================
    # RETORNO DAS FEATURES / FEATURE OUTPUT
    # ================================
    return pd.DataFrame([{
        "block_height": block_height,
        "block_timestamp": block_timestamp,
        "anova_p_mes": p_mes,
        "anova_p_ano": p_ano,
        "anova_p_interacao": p_inter
    }])
    # --> Retorna como linha única com chaves de rastreio / Returns row with tracking keys <--

In [ ]:
df_seasonality = extract_seasonal_anova_features(df_seasonal)
display(df_seasonality)

#### 4.3 Analise Espectral (FFT)

In [ ]:
# Para df_fft
df_fft = df_time_index.resample('d').mean()
df_fft = df_fft.sort_index()  # Garantir que está ordenado pelo índice (block_timestamp)

##### 🔶 ₿ -----> Frequência Dominante

---

Transformada Rápida de Fourier (FFT)

Dada uma série temporal discreta $ x[n] $ de tamanho $ N $, sua transformada discreta de Fourier é definida por:

$
X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j2\pi kn/N}
$

Onde:

- $ X[k] $: componente de frequência no índice $( k $)
- $ N $: número total de pontos na série
- $ j $: unidade imaginária $( j^2 = -1 )$

A frequência correspondente a cada índice \( k \) é dada por:

$
f[k] = \frac{k}{N}
$

Para obter as **frequências positivas** e suas **amplitudes** (magnitudes):

- Frequências positivas:  
  $
  f = \text{fft\_freqs}[f > 0]
  $

- Magnitudes associadas:  
  $
  |X[k]| = \sqrt{\Re(X[k])^2 + \Im(X[k])^2}
  $

---

Identificação da Frequência Dominante

A **frequência dominante** é aquela cujo valor de \( |X[k]| \) (amplitude) é o **máximo** dentre todas as componentes positivas:

$
f_{\text{dom}} = f[argmax(|X[k]|)]
$

---

In [ ]:
# ================================================================
# PREPARAÇÃO DA SÉRIE PARA FFT / PREPARE SERIES FOR FFT
# ================================================================
def prepare_fft_data(series: pd.Series):
    """
    Pré-processa a série e retorna as frequências e amplitudes positivas.
    / Preprocesses the series and returns positive frequencies and magnitudes.
    """

    series = series.dropna().values
    # --> Remove valores ausentes e converte para array NumPy / Drops missing values and converts to NumPy array <--

    n = len(series)
    # --> Define o comprimento da série para cálculo da FFT / Defines series length for FFT calculation <--

    fft_vals = np.fft.fft(series)
    # --> Calcula os coeficientes da Transformada de Fourier / Computes the Fourier transform coefficients <--

    fft_freqs = np.fft.fftfreq(n)
    # --> Gera a grade de frequências correspondente / Generates the corresponding frequency grid <--

    pos_mask = fft_freqs > 0
    # --> Cria uma máscara para manter apenas frequências positivas / Creates mask to retain only positive frequencies <--

    freqs = fft_freqs[pos_mask]
    # --> Frequências positivas / Positive frequencies <--

    magnitudes = np.abs(fft_vals[pos_mask])
    # --> Magnitudes (amplitudes absolutas) das componentes positivas / Absolute magnitudes of positive components <--

    return freqs, magnitudes
    # --> Retorna as frequências e magnitudes para análise espectral / Returns frequencies and magnitudes for spectral analysis <--

In [ ]:
freqs, magnitudes = prepare_fft_data(df_fft["btc_price_usd"])

print("Frequências detectadas:", freqs[:5])
print("Magnitudes:", magnitudes[:5])

##### 🔶 ₿ -----> Energia Espectral

---

Energia Espectral e Razão de Energia

Seja uma série temporal transformada via FFT, com magnitudes espectrais $ M_k $, a **energia espectral total** da série é dada por:

$
E_{\text{total}} = \sum_{k=1}^{N} M_k^2
$

Onde:

- $ M_k $: magnitude da componente de frequência $ k $
- $ N $: número total de componentes de frequência

A **energia top‑K** corresponde à soma das $ K $ maiores contribuições de energia:

$
E_{\text{top‑K}} = \sum_{k \in \text{Top‑K}} M_k^2
$

A **razão de energia espectral** é definida como:

$
\text{Razão} = \frac{E_{\text{top‑K}}}{E_{\text{total}}}
$

---

In [ ]:
# ================================================================
# CÁLCULO DA RAZÃO DE ENERGIA / ENERGY RATIO
# ================================================================
def calculate_energy_ratio(magnitudes: np.ndarray, top_k: int):
    """
    Calcula a razão entre a energia das top-k amplitudes e a energia total.
    / Calculates the ratio between top-k dominant energy and total energy.
    """

    total_energy = np.sum(magnitudes**2)
    # --> Calcula a energia total da série (soma dos quadrados das magnitudes) / Computes total energy (sum of squared magnitudes) <--

    topk_energy = np.sum(np.sort(magnitudes**2)[-top_k:])
    # --> Soma da energia das top-k frequências dominantes / Sum of energy from top-k dominant frequencies <--

    return topk_energy / total_energy
    # --> Retorna a razão de energia concentrada nas top-k componentes / Returns energy ratio of dominant frequencies <--

In [ ]:
energy_ratio = calculate_energy_ratio(magnitudes, top_k=3)
print("Razão de energia espectral:", energy_ratio)

##### 🔶 ₿ -----> Entropia Espectral

---

Entropia Espectral (Spectral Entropy)

A **entropia espectral** quantifica o grau de desordem ou dispersão da energia em diferentes frequências de uma série temporal. Ela é baseada na distribuição normalizada das magnitudes do espectro de Fourier.

---

Definição formal:

Seja $ M_k $ a magnitude da componente de frequência $ k $, a **distribuição de probabilidade espectral** $ P_k $ é definida como:

$
P_k = \frac{M_k}{\sum_{i=1}^{N} M_i}
$

A entropia espectral é então dada por:

$
H = - \sum_{k=1}^{N} P_k \cdot \log_2(P_k)
$

> Obs: Na prática, adiciona-se um termo $ \varepsilon $ muito pequeno para evitar log de zero:  
$
H = - \sum_{k=1}^{N} P_k \cdot \log_2(P_k + \varepsilon)
\quad \text{com } \varepsilon = 10^{-12}
$

---

Interpretação:

- Baixa entropia espectral: concentração de energia em poucas frequências → sinal mais previsível ou periódico.
- Alta entropia espectral: energia distribuída em muitas frequências → sinal mais complexo ou ruído branco.

---



In [ ]:
# ================================================================
# CÁLCULO DA ENTROPIA ESPECTRAL / SPECTRAL ENTROPY
# ================================================================
def calculate_spectral_entropy(magnitudes: np.ndarray):
    """
    Calcula a entropia espectral da distribuição de frequência.
    / Computes the spectral entropy of the frequency distribution.
    """

    prob_dist = magnitudes / np.sum(magnitudes)
    # --> Converte magnitudes em distribuição de probabilidade / Converts magnitudes into a probability distribution <--

    return -np.sum(prob_dist * np.log2(prob_dist + 1e-12))
    # --> Aplica a fórmula da entropia de Shannon / Applies Shannon entropy formula <--

In [ ]:
# ================================================================
# FUNÇÃO PRINCIPAL / MAIN FEATURE EXTRACTION FUNCTION
# ================================================================
def extract_fft_features(series: pd.Series, top_k: int = 3) -> dict:
    """
    Extrai features estruturais da série temporal via FFT.
    / Extracts structural features from time series using FFT.
    """

    freqs, magnitudes = prepare_fft_data(series)
    # --> Pré-processa a série e obtém frequências e magnitudes positivas / Preprocesses the series and gets positive frequencies and magnitudes <--

    return {
        "fft_dominant_freq": freqs[np.argmax(magnitudes)],
        # --> Frequência com maior magnitude (componente dominante) / Frequency with highest magnitude (dominant component) <--

        "fft_energy_ratio": calculate_energy_ratio(magnitudes, top_k),
        # --> Razão de energia concentrada nas top-k componentes / Energy ratio from top-k dominant components <--

        "fft_peak_amplitude": np.max(magnitudes),
        # --> Valor da amplitude de pico entre as frequências / Peak amplitude value among all frequencies <--

        "fft_spectral_entropy": calculate_spectral_entropy(magnitudes)
        # --> Entropia espectral (grau de desorganização da energia) / Spectral entropy (energy dispersion measure) <--
    }

In [ ]:
entropy = calculate_spectral_entropy(magnitudes)
print("Entropia espectral:", entropy)

##### 🔶 ₿ -----> Resultado Geral das Frequências Dominantes e Entropia

In [ ]:
features_fft = extract_fft_features(df_fft["btc_price_usd"], top_k=3)

for k, v in features_fft.items():
    print(f"{k}: {v}")
    
period_in_steps = 1 / features_fft["fft_dominant_freq"]
print(f" Ciclo dominante estimado: {period_in_steps:.2f} passos")

features_fft

##### 🔶 ₿ -----> Features

In [ ]:

# ================================================================
# PRÉ-PROCESSAMENTO PARA FFT / PREPROCESSING FOR FFT
# ================================================================
def prepare_fft_data(series: pd.Series):
    """
    Pré-processa a série e retorna frequências e magnitudes positivas.
    / Preprocesses the series and returns positive frequencies and magnitudes.
    """
    series = series.dropna().values
    n = len(series)
    fft_vals = np.fft.fft(series)
    fft_freqs = np.fft.fftfreq(n)

    pos_mask = fft_freqs > 0
    # --> Considera apenas frequências positivas / Keep only positive frequencies <--

    freqs = fft_freqs[pos_mask]
    magnitudes = np.abs(fft_vals[pos_mask])
    # --> Calcula módulo das componentes espectrais / Compute magnitude of spectral components <--

    return freqs, magnitudes

# ================================================================
# CÁLCULO DA RAZÃO DE ENERGIA / ENERGY RATIO
# ================================================================
def calculate_energy_ratio(magnitudes: np.ndarray, top_k: int) -> float:
    """
    Calcula a razão entre a energia das top-k amplitudes e a energia total.
    / Calculates the ratio between top-k dominant energy and total energy.
    """
    total_energy = np.sum(magnitudes**2)
    # --> Energia total = soma dos quadrados das magnitudes / Total energy = sum of squared magnitudes <--

    topk_energy = np.sum(np.sort(magnitudes**2)[-top_k:])
    # --> Energia nas k maiores componentes / Energy from top-k dominant components <--

    return topk_energy / total_energy
    # --> Retorna a razão de concentração energética / Returns energy concentration ratio <--

# ================================================================
# CÁLCULO DA ENTROPIA ESPECTRAL / SPECTRAL ENTROPY
# ================================================================
def calculate_spectral_entropy(magnitudes: np.ndarray) -> float:
    """
    Calcula a entropia espectral da distribuição de frequência.
    / Computes the spectral entropy of the frequency distribution.
    """
    prob_dist = magnitudes / np.sum(magnitudes)
    # --> Distribuição de probabilidade normalizada / Normalized probability distribution <--

    return -np.sum(prob_dist * np.log2(prob_dist + 1e-12))
    # --> Entropia de Shannon (com estabilidade numérica) / Shannon entropy (with numerical stability) <--

# ================================================================
# EXTRAÇÃO DE FEATURES FFT COM JANELAS MÓVEIS
# ================================================================
def extract_fft_features_df(
    series: pd.Series,
    window_size: int = 60,
    step: int = 10,
    top_k: int = 3
) -> pd.DataFrame:
    """
    Extrai features estruturais da série via FFT e retorna um DataFrame com múltiplas janelas móveis.
    / Extracts structural features from time series using FFT and returns a DataFrame with multiple rolling windows.

    Parâmetros / Parameters:
    - series: pd.Series
        Série temporal com índice temporal / Time-indexed numeric series.
    - window_size: int
        Tamanho da janela móvel (número de pontos) / Size of the rolling window (number of points).
    - step: int
        Passo entre janelas consecutivas / Step size between consecutive windows.
    - top_k: int
        Número de componentes principais de frequência / Number of dominant frequency components.

    Retorna / Returns:
    - pd.DataFrame com features para todas as janelas móveis / DataFrame with features for all rolling windows.
    """
    result_list = []

    for i in range(0, len(series) - window_size + 1, step):
        window = series.iloc[i:i + window_size]

        # Dados de rastreio
        block_height = series.index[i + window_size - 1]
        block_timestamp = series.index[i + window_size - 1]

        # Processamento de FFT
        freqs, magnitudes = prepare_fft_data(window)

        # Extração das features
        features = {
            "block_height": block_height,
            "block_timestamp": block_timestamp,
            "fft_dominant_freq": freqs[np.argmax(magnitudes)],
            # --> Frequência dominante (maior pico) / Dominant frequency (highest peak) <--

            "fft_energy_ratio": calculate_energy_ratio(magnitudes, top_k),
            # --> Razão da energia nas top-k frequências / Energy ratio in top-k components <--

            "fft_peak_amplitude": np.max(magnitudes),
            # --> Amplitude máxima / Maximum amplitude <--

            "fft_spectral_entropy": calculate_spectral_entropy(magnitudes)
            # --> Entropia espectral / Spectral entropy <--
        }

        # Adiciona os resultados para cada janela
        result_list.append(features)

    # Retorna todos os resultados concatenados em um DataFrame
    return pd.DataFrame(result_list)
    # --> Retorna as features para todas as janelas móveis / Returns features for all rolling windows <--

In [ ]:
# Definindo parâmetros de janela e passo
window_size = 60  # Tamanho da janela (exemplo: 60 períodos de dados)
step = 10         # Passo de 10 períodos entre as janelas

# Chamando a função para extrair as features FFT para a série temporal
fft_features_df = extract_fft_features_df(series=df_fft['btc_price_usd'], window_size=window_size, step=step)

# Exibindo as primeiras 5 linhas do DataFrame resultante
display(fft_features_df.head())

In [ ]:
df_peaks = extract_peak_features(df_fft, column="btc_price_usd", distance=8)
display(df_peaks.head())

In [ ]:
# Janela de exemplo
window = df_fft["btc_price_usd"].iloc[-60:]
block_height_final = df_fft["block_height"].iloc[-1]
timestamp_final = df_fft["block_timestamp"].iloc[-1]

# Execução
df_ciclico = extract_cyclic_features(
    series=window,
    block_height=block_height_final,
    block_timestamp=timestamp_final,
    distance=8
)

display(df_ciclico)

In [ ]:
series = df_fft["btc_price_usd"].dropna()
# --> Remove valores ausentes da série / Drops missing values from the series <--

n = len(series)
# --> Número total de observações na série / Total number of observations in the series <--

fft_vals = np.fft.fft(series)
# --> Aplica a Transformada Rápida de Fourier (FFT) / Applies Fast Fourier Transform (FFT) <--

fft_freqs = np.fft.fftfreq(n)
# --> Frequências associadas aos coeficientes FFT / Frequencies corresponding to FFT coefficients <--

# ===========================
# FILTRO: FREQUÊNCIAS POSITIVAS
# ===========================

mask = fft_freqs > 0
fft_freqs = fft_freqs[mask]
fft_vals = fft_vals[mask]
# --> Mantém apenas as frequências positivas (metade direita do espectro) / Keeps only positive frequencies (right half of the spectrum) <--

# ================================
# CÁLCULO DA POTÊNCIA / POWER SPECTRUM
# ================================

power = np.abs(fft_vals)**2
# --> Potência espectral (amplitude²) de cada componente harmônico / Spectral power (amplitude²) for each harmonic <--

periods = 1 / fft_freqs
# --> Converte frequência em período (em unidades de tempo) / Converts frequency to period (in time units) <--

# ===================================
# FILTRAGEM DE PERÍODOS EXTREMOS
# ===================================

valid = (periods < 500)
periods_filtered = periods[valid]
power_filtered = power[valid]
# --> Remove períodos muito longos que não são relevantes (ex: acima de 500 unidades) / Filters out long periods (e.g., over 500 units) <--

# ===================================
# SELEÇÃO DOS PRINCIPAIS PONTOS
# ===================================

top_peaks = np.argsort(power_filtered)[-3:]
# --> Seleciona os índices das 3 maiores potências (frequências dominantes) / Selects indices of the top 3 dominant frequencies <--

In [ ]:
plot_fft_spectrum(
    periods=periods_filtered,
    power=power_filtered,
    fft_vals=fft_vals,
    n=n,
    top_peaks_idx=top_peaks,
    image_path="/Users/rodrigocampos/Documents/Bitcoin/project/src/visualizations/BTC_black.png"
)

##### 🔶 ₿ -----> Reconstrução Cíclica com FFT

In [ ]:
reconstructed = reconstruct_fft(df_fft["btc_price_usd"], 3)

In [ ]:
# Série de preços original
price_series = df_stl["btc_price_usd"]
# --> Série original de preços / Original price series <--

# Série centralizada (remoção da tendência via STL ou suavização)
price_detrended = price_series - df_stl_dec["trend"]
# --> Remove a tendência da série para aplicar a FFT / Removes trend to isolate cyclical components <--

In [ ]:
# ==========================
# PLOTAGEM DA RECONSTRUÇÃO FFT
# ==========================

fig_fft_recon = go.Figure()

# --- Série Reconstruída com as principais frequências (FFT) ---
fig_fft_recon.add_trace(go.Scatter(
    x=df_fft.index,
    y=reconstructed,
    name="Reconstrução FFT (Top Frequências)",
    line=dict(color="white", width=2, dash="dot"),
    fill="tozeroy",
    fillcolor="rgba(229,165,0,0.25)"
))
# --> Linha pontilhada branca representando os ciclos reconstruídos via FFT / Dashed white line: cycles reconstructed using FFT <--

# --- Série original centralizada (sem tendência) ---
fig_fft_recon.add_trace(go.Scatter(
    x=df_fft.index,
    y=price_detrended,
    name="Série Centralizada",
    line=dict(color="#E57C1F", width=1.2),
    opacity=0.3
))
# --> Série de preços original com tendência removida (centralizada) / Original series with trend removed (centered) <--

# ==========================
# LAYOUT DO GRÁFICO
# ==========================

fig_fft_recon.update_layout(
    title="<b><span style='font-size:22px;'>Reconstrução Cíclica com FFT</span><br><span style='font-size:14px;'>Com base nas principais frequências da série</span></b>",
    xaxis_title="Data",
    yaxis_title="Amplitude (Centralizada)",
    template="plotly_dark",
    height=500,
    showlegend=True
)
# --> Título, eixos e estilo escuro com preenchimento harmônico / Title, axis, and dark theme with FFT reconstruction fill <--

fig_fft_recon.show()
# --> Exibe o gráfico final interativo / Displays the final interactive chart <--

#### 4.4 Detecção de Padrões Cíclicos

In [ ]:


# ================================================================
# DETECÇÃO DE PADRÕES CÍCLICOS / CYCLE PATTERN DETECTION
# ================================================================

peaks, _ = find_peaks(series, distance=10)
# --> Encontra os índices dos picos com distância mínima entre eles / Finds peak indices with a minimum spacing <--

vales, _ = find_peaks(-series, distance=10)
# --> Inverte o sinal da série para detectar mínimos como picos / Inverts signal to find valleys as peaks <--

In [ ]:
ciclo_duracoes = np.diff(vales)
amplitudes = series[peaks] - series[vales[:len(peaks)]]

#### 4.5 Estrutura e Persistência Temporal

Coeficiente de Hurst (Memória Longa)

O **coeficiente de Hurst** $ H $ é uma medida de **persistência de longo prazo** de uma série temporal. Ele indica se a série tende a:

- Reverter à média: $ H < 0.5 $
- Ser aleatória (ruído branco): $ H \approx 0.5 $
- Persistir na direção atual: $ H > 0.5 $

O cálculo baseia-se no crescimento da variância com o tempo:

$
\text{Var}(X_t) \propto t^{2H}
$

---

Número de Cruzamentos com a Mediana

Esta métrica indica **quantas vezes a série cruza sua mediana**. Um número alto sugere comportamento mais **oscilatório**, enquanto um número baixo sugere **persistência ou tendência**.

$
\text{crossings} = \sum_{t=2}^{n} \mathbf{1}\{(x_{t-1} - \text{mediana}) \cdot (x_t - \text{mediana}) < 0\}
$

---

Maior Região de Estagnação

Mede o maior segmento contínuo da série com **baixa variação**, representando **zonas de lateralização** ou ausência de movimento relevante.

$
\text{flat\_spot} = \max\{\text{duração de regiões quase constantes}\}
$

Esses três recursos ajudam a quantificar o grau de **regularidade, memória e oscilação** da série, podendo ser úteis para detectar **comportamentos anômalos ou regimes de mercado**.

---

In [ ]:


# ================================================================
# COEFICIENTE DE HURST / HURST EXPONENT
# ================================================================
def compute_hurst(series: np.ndarray) -> float:
    """
    Calcula o coeficiente de Hurst (memória longa).
    / Computes the Hurst exponent (long memory indicator).
    """
    series = series[~np.isnan(series)]
    # --> Remove valores ausentes / Remove NaN values <--

    H, _, _ = compute_Hc(series, kind='price')
    # --> Estima o coeficiente de Hurst usando o método 'price' / Estimates Hurst exponent with 'price' method <--

    return H
    # --> Retorna o valor estimado de H / Returns the estimated Hurst exponent <--

In [ ]:
# ================================================================
# CRUZAMENTOS COM A MEDIANA / MEDIAN CROSSINGS
# ================================================================
def count_median_crossings(series: np.ndarray) -> int:
    """
    Conta quantos cruzamentos com a mediana ocorrem na série.
    / Counts how many times the series crosses its median.
    """
    median_val = np.median(series)
    # --> Calcula a mediana da série / Computes series median <--

    shifted = (series - median_val) > 0
    # --> Converte a série para sinais positivos e negativos / Converts series to binary signal above/below median <--

    return np.sum(shifted[1:] != shifted[:-1])
    # --> Conta mudanças de sinal (cruzamentos) / Counts sign changes (crossings) <--

In [ ]:
# ================================================================
# MAIOR REGIÃO DE ESTAGNAÇÃO / LONGEST FLAT SPOT
# ================================================================
def longest_flat_spot(series: np.ndarray, tol=1e-6) -> int:
    """
    Encontra a maior sequência de variação mínima (estagnação).
    / Finds the longest sequence with minimal variation (stagnation).
    """
    diff = np.abs(np.diff(series))
    # --> Calcula a diferença absoluta entre observações consecutivas / Absolute difference between consecutive values <--

    flat = diff < tol
    # --> Marca onde a variação é menor que a tolerância / Marks where variation is below tolerance <--

    max_len = count = 0
    # --> Inicializa contadores / Initializes counters <--

    for val in flat:
        count = count + 1 if val else 0
        max_len = max(max_len, count)
    # --> Atualiza o comprimento máximo de sequência plana / Updates max length of flat sequence <--

    return max_len
    # --> Retorna o comprimento da maior região de estagnação / Returns length of longest stagnation segment <--

In [ ]:


# ==========================================================
# Cálculo do Coeficiente de Hurst com fallback para séries curtas
# Compute Hurst Exponent with fallback for short series
# ==========================================================
def compute_hurst(series, min_length=100):
    """
    Calcula o coeficiente de Hurst se a série tiver pelo menos `min_length` pontos
    Computes the Hurst exponent if the series has at least `min_length` points
    """
    series = np.array(series)
    series = series[~np.isnan(series)]
    # --> Remove valores ausentes / Remove NaN values <--

    if len(series) < min_length:
        print(f"[AVISO] Série com apenas {len(series)} pontos. Retornando np.nan.")
        # --> Retorna NaN se a série for muito curta / Returns NaN if the series is too short <--
        return np.nan

    try:
        H, _, _ = compute_Hc(series, kind='price')
        # --> Estima o coeficiente de Hurst usando o método 'price' / Estimates Hurst exponent using 'price' method <--
        return H
    except Exception as e:
        print(f"[ERRO] Falha ao calcular Hurst: {e}")
        # --> Retorna NaN em caso de erro / Returns NaN if Hurst computation fails <--
        return np.nan

# ==========================================================
# Contagem de cruzamentos com a mediana
# Count crossings with the median
# ==========================================================
def count_median_crossings(series):
    median = np.median(series)
    # --> Calcula a mediana da série / Computes the median of the series <--

    shifted = np.roll(series, 1)
    # --> Desloca a série para comparar com o valor anterior / Shifts the series to compare with the previous value <--

    crossings = (np.sign(series - median) != np.sign(shifted - median)).astype(int)
    # --> Detecta mudança de sinal em relação à mediana / Detects sign change with respect to the median <--

    return np.sum(crossings[1:])
    # --> Soma os cruzamentos, ignorando o primeiro valor deslocado / Sums the crossings, ignoring the first shifted value <--

# ==========================================================
# Maior sequência com variação abaixo do limite (flat spot)
# Longest low-variation sequence (flat spot)
# ==========================================================
def longest_flat_spot(series, tolerance=1e-5):
    diffs = np.abs(np.diff(series))
    # --> Calcula a diferença absoluta entre elementos consecutivos / Calculates absolute difference between consecutive elements <--

    flat = diffs < tolerance
    # --> Marca os pontos com variação muito baixa / Marks points with very low variation <--

    max_len = count = 0
    for val in flat:
        count = count + 1 if val else 0
        max_len = max(max_len, count)

    return max_len + 1 if max_len > 0 else 0
    # --> Retorna o comprimento máximo da sequência plana / Returns the max length of flat region <--

# ==========================================================
# APLICAÇÃO NA SÉRIE / APPLY TO YOUR SERIES
# ==========================================================
series_np = df_fft["btc_price_usd"].dropna().values
# --> Converte a série para array NumPy e remove NaNs / Converts series to NumPy array and drops NaNs <--

hurst_val = compute_hurst(series_np)
# --> Calcula o coeficiente de Hurst / Computes Hurst exponent <--

crossings = count_median_crossings(series_np)
# --> Conta os cruzamentos com a mediana / Counts median crossings <--

flat_len = longest_flat_spot(series_np)
# --> Calcula a maior sequência com baixa variação / Computes longest flat spot <--

print("Coeficiente de Hurst:", hurst_val)
# --> Exibe o coeficiente de Hurst / Prints Hurst exponent <--

print("Cruzamentos com a mediana:", crossings)
# --> Exibe a contagem de cruzamentos / Prints median crossing count <--

print("Maior região de estagnação:", flat_len)
# --> Exibe o tamanho da maior sequência estagnada / Prints longest flat segment length <--

In [ ]:
# ================================================================
# CONSTRUÇÃO DO DATAFRAME DE PERSISTÊNCIA / PERSISTENCE FEATURES
# ================================================================

df_persistence = pd.DataFrame([{
    "hurst_exponent": hurst_val,
    "median_crossings": crossings,
    "longest_flat_spot": flat_len
}])
# --> Cria DataFrame com as features de persistência extraídas / Creates DataFrame with extracted persistence features <--

# ================================
# ADIÇÃO DAS CHAVES TEMPORAIS
# ADDING TIMESTAMP AND BLOCK HEIGHT KEYS
# ================================
df_persistence["block_timestamp"] = df_fft["block_timestamp"].iloc[-1]
# --> Adiciona o timestamp da última janela usada / Adds timestamp of the last window used <--

df_persistence["block_height"] = df_fft["block_height"].iloc[-1]
# --> Adiciona a altura do bloco correspondente / Adds corresponding block height <--

# ================================
# CONSTRUÇÃO DO DATAFRAME FINAL
# FINAL DISPLAY
# ================================
display(df_persistence)

4.6 Estabilidade Local e Heterocedasticidade

---

Variância de Blocos (var_tiled_var)

Divide a série em blocos consecutivos e calcula a variância dentro de cada bloco. A **variabilidade dessas variâncias** indica **instabilidade ao longo do tempo**.

$
\text{VarBloco}_i = \text{Var}(X_{i1}, X_{i2}, ..., X_{im})
$

---

Média de Blocos (var_tiled_mean)

Calcula a média de cada bloco e depois mede a **variância entre essas médias**. É útil para identificar **mudanças no nível médio da série ao longo do tempo**.

$
\text{MeanBloco}_i = \frac{1}{m} \sum_{j=1}^{m} X_{ij}
\quad \Rightarrow \quad \text{Var}\left( \text{MeanBloco}_i \right)
$

---

Teste ARCH (Engle LM)

O **teste ARCH** verifica se os resíduos têm **heterocedasticidade condicional**, ou seja, **variância que muda ao longo do tempo**.  
É usado para avaliar se modelos como **GARCH** são apropriados.

$
\text{H}_0: \text{Não há efeito ARCH (variância constante)}
\quad \text{vs} \quad
\text{H}_1: \text{Existe heterocedasticidade condicional}
$

Resultado:

- **Estatística LM**
- **Valor-p** (se < 0.05, rejeita-se $ H_0 $)

---

In [ ]:
# ================================================================
# VARIÂNCIA DAS VARIÂNCIAS POR BLOCO / VARIANCE OF BLOCK-WISE VARIANCES
# ================================================================
def var_tiled_var(series: np.ndarray, block_size: int = 10) -> float:
    """
    Variância das variâncias por blocos / Variance of block-wise variances.
    """
    if isinstance(series, pd.Series):
        series = series.dropna().values
    else:
        series = series[~np.isnan(series)]
    # --> Garante array NumPy e remove NaNs / Ensures NumPy array and drops NaNs <--

    trimmed = series[:len(series) // block_size * block_size]
    # --> Ajusta o tamanho da série para ser divisível pelo tamanho do bloco / Trims series to be divisible by block size <--

    blocks = trimmed.reshape(-1, block_size)
    # --> Reshape da série em blocos não sobrepostos / Reshapes series into non-overlapping blocks <--

    block_vars = np.var(blocks, axis=1, ddof=1)
    # --> Calcula a variância de cada bloco / Computes variance within each block <--

    return np.var(block_vars, ddof=1)
    # --> Retorna a variância das variâncias dos blocos / Returns variance of block-wise variances <--

In [ ]:
# ================================================================
# VARIÂNCIA DAS MÉDIAS POR BLOCO / VARIANCE OF BLOCK-WISE MEANS
# ================================================================
def var_tiled_mean(series: np.ndarray, block_size: int = 10) -> float:
    """
    Variância das médias por blocos / Variance of block-wise means.
    """
    if isinstance(series, pd.Series):
        series = series.dropna().values
    else:
        series = series[~np.isnan(series)]
    # --> Garante array NumPy e remove NaNs / Ensures NumPy array and drops NaNs <--

    trimmed = series[:len(series) // block_size * block_size]
    # --> Ajusta o comprimento para ser divisível pelo tamanho do bloco / Trims to fit block size <--

    blocks = trimmed.reshape(-1, block_size)
    # --> Divide em blocos consecutivos não sobrepostos / Reshapes into consecutive blocks <--

    block_means = np.mean(blocks, axis=1)
    # --> Calcula a média de cada bloco / Computes mean of each block <--

    return np.var(block_means, ddof=1)
    # --> Retorna a variância entre as médias dos blocos / Returns variance of block-wise means <--

In [ ]:
# ================================================================
# TESTE DE HETEROCEDASTICIDADE ARCH / ARCH EFFECT TEST (ENGLE LM)
# ================================================================
def arch_test(series: np.ndarray, lags: int = 12) -> dict:
    """
    Teste de heterocedasticidade ARCH (Engle LM).
    / ARCH effect test using Engle's Lagrange Multiplier.
    """
    series = series[~np.isnan(series)]
    # --> Remove valores ausentes / Remove NaNs <--

    lm_stat, lm_pvalue, _, _ = het_arch(series, nlags=lags)
    # --> Executa o teste LM com número definido de lags / Runs ARCH LM test with defined lags <--

    return {
        "arch_lm_stat": lm_stat,
        # --> Estatística do teste LM / LM test statistic <--
        "arch_pvalue": lm_pvalue
        # --> Valor-p do teste / p-value from the test <--
    }

In [ ]:
# ================================================================
# FUNÇÃO AUXILIAR: VARIÂNCIA ENTRE MÉDIAS DE BLOCOS
# AUXILIARY FUNCTION: VARIANCE ACROSS BLOCK MEANS
# ================================================================
def var_tiled_mean(series, block_size):
    series = np.array(series)
    series = series[~np.isnan(series)]
    # --> Remove NaNs / Drop NaNs <--

    if len(series) < block_size:
        return np.nan
        # --> Série muito curta para blocos / Too short for block splitting <--

    num_blocks = len(series) // block_size
    blocks = series[:num_blocks * block_size].reshape(num_blocks, block_size)
    block_means = blocks.mean(axis=1)
    return np.var(block_means)


# ================================================================
# FUNÇÃO AUXILIAR: VARIÂNCIA ENTRE VARIÂNCIAS DE BLOCOS
# AUXILIARY FUNCTION: VARIANCE ACROSS BLOCK VARIANCES
# ================================================================
def var_tiled_var(series, block_size):
    series = np.array(series)
    series = series[~np.isnan(series)]
    # --> Remove NaNs / Drop NaNs <--

    if len(series) < block_size:
        return np.nan
        # --> Série muito curta para blocos / Too short for block splitting <--

    num_blocks = len(series) // block_size
    blocks = series[:num_blocks * block_size].reshape(num_blocks, block_size)
    block_vars = blocks.var(axis=1)
    return np.var(block_vars)


# ================================================================
# FUNÇÃO AUXILIAR: TESTE ARCH COM PROTEÇÃO
# AUXILIARY FUNCTION: ARCH TEST WITH VALIDATION
# ================================================================
def arch_test(series, lags):
    """
    Executa o teste ARCH se houver observações suficientes.
    Runs ARCH test if series is long enough.
    """
    if len(series) <= lags:
        print(f"[AVISO] Série com {len(series)} pontos < número de lags ({lags}). ARCH não será executado.")
        return {"arch_lm_stat": np.nan, "arch_pvalue": np.nan}
        # --> Retorna NaNs se série for curta / Returns NaNs for short series <--

    try:
        arch_lm_stat, arch_pvalue, _, _ = het_arch(series, maxlag=lags)
        return {"arch_lm_stat": arch_lm_stat, "arch_pvalue": arch_pvalue}
        # --> Executa o teste e retorna estatísticas / Runs ARCH test and returns stats <--
    except Exception as e:
        print(f"[ERRO] Falha no teste ARCH: {e}")
        return {"arch_lm_stat": np.nan, "arch_pvalue": np.nan}
        # --> Em caso de erro, retorna NaNs / On error, returns NaNs <--

# ================================================================
# EXTRAÇÃO DE FEATURES DE VOLATILIDADE / VOLATILITY FEATURE EXTRACTION
# ================================================================
def extract_volatility_features(
    series: pd.Series,
    block_height: int,
    block_timestamp: pd.Timestamp,
    block_size: int = 10,
    arch_lags: int = 12
) -> pd.DataFrame:
    """
    Extrai features de estabilidade local e heterocedasticidade.
    / Extracts local variance and heteroskedasticity features.

    Parâmetros / Parameters:
    - series: pd.Series ou np.ndarray
        Série temporal numérica / Numeric time series.
    - block_height: int
        Altura do bloco associada à janela / Block height for tracking.
    - block_timestamp: pd.Timestamp
        Timestamp associado à janela / Timestamp for tracking.
    - block_size: int
        Tamanho dos blocos para análise local / Block size for local analysis.
    - arch_lags: int
        Número de lags para o teste ARCH / Number of lags for ARCH test.

    Retorna / Returns:
    - pd.DataFrame com uma linha contendo: block_height, block_timestamp, var_tiled_mean, var_tiled_var, arch_lm_stat, arch_pvalue
    / One-row DataFrame with volatility and heteroskedasticity features.
    """

    # ===========================
    # PREPARAÇÃO DA SÉRIE / SERIES PREPROCESSING
    # ===========================
    if isinstance(series, pd.Series):
        series = series.dropna().values
        # --> Converte pd.Series para array e remove NaNs / Convert Series to array and drop NaNs <--
    else:
        series = series[~np.isnan(series)]
        # --> Remove NaNs de array NumPy / Drop NaNs from NumPy array <--

    # ===========================
    # CÁLCULO DAS FEATURES / FEATURE EXTRACTION
    # ===========================
    features = {
        "var_tiled_mean": var_tiled_mean(series, block_size),
        # --> Variância entre médias de blocos / Variance across block means <--

        "var_tiled_var": var_tiled_var(series, block_size),
        # --> Variância entre variâncias de blocos / Variance across block variances <--

        **arch_test(series, lags=arch_lags)
        # --> Estatísticas do teste ARCH / ARCH test metrics <--
    }

    # ===========================
    # FORMATAÇÃO FINAL / FINAL OUTPUT
    # ===========================
    features["block_height"] = block_height
    features["block_timestamp"] = block_timestamp
    # --> Adiciona metadados para rastreio / Adds tracking metadata <--

    return pd.DataFrame([features])
    # --> Retorna resultado como DataFrame de uma linha / Returns one-row DataFrame with features <--

In [ ]:
window = df_returns["log_return"].iloc[-96:]
block_height_ref = df_returns["block_height"].iloc[-1]
block_timestamp_ref = df_returns["block_timestamp"].iloc[-1]

df_vol_features = extract_volatility_features(
    series=window,
    block_height=block_height_ref,
    block_timestamp=block_timestamp_ref,
    block_size=10,
    arch_lags=12
)
display(df_vol_features)

4.7 Mudanças Estruturais e Quebras

---


Mudança de Nível (shift_level_max)

Compara **janelas consecutivas da série** e identifica a maior diferença entre suas médias. É usada para encontrar **saltos ou quedas abruptas** na tendência.

$
\text{shift\_level}_t = \left| \text{média}(X_{t:t+w}) - \text{média}(X_{t+w:t+2w}) \right|
$

---

Mudança de Variância (shift_var_max)

Similar à anterior, mas avalia a **diferença na variância** entre duas janelas consecutivas, revelando mudanças de **volatilidade**.

$
\text{shift\_var}_t = \left| \text{var}(X_{t:t+w}) - \text{var}(X_{t+w:t+2w}) \right|
$

---

Mudança de Distribuição (KL Divergence)

Utiliza a **divergência de Kullback-Leibler** para quantificar o quão diferente é a distribuição de duas janelas consecutivas. Quanto maior, maior a **ruptura distribucional**.

$
\text{KL}(P || Q) = \sum_i P(i) \log \left( \frac{P(i)}{Q(i)} \right)
$

- $ P $: distribuição na janela atual  
- $ Q $: distribuição na próxima janela  
- Resultado: índice do maior KL + valor máximo

---

In [ ]:
# ================================================================
# DIVERGÊNCIA DE KULLBACK-LEIBLER / KULLBACK-LEIBLER DIVERGENCE
# ================================================================
def kl_divergence(p, q):
    """
    Calcula a divergência de KL entre duas distribuições.
    / Computes the KL divergence between two distributions.
    """
    p = np.asarray(p) + 1e-12
    q = np.asarray(q) + 1e-12
    # --> Garante que não há divisão por zero / Ensures no division by zero <--

    return np.sum(p * np.log(p / q))
    # --> Soma da divergência ponto a ponto / Sum of pointwise divergence <--

# ================================================================
# MUDANÇAS ESTRUTURAIS NA SÉRIE TEMPORAL / STRUCTURAL SHIFTS IN TIME SERIES
# ================================================================
def extract_structural_shifts(
    series: pd.Series,
    block_height: int,
    block_timestamp: pd.Timestamp,
    window: int = 30,
    bins: int = 20
) -> pd.DataFrame:
    """
    Detecta rupturas estruturais usando média, variância e divergência KL.
    / Detects structural shifts using mean, variance, and KL divergence.
    """

    series = series.dropna().values
    # --> Remove valores ausentes e converte para array / Clean and convert to NumPy array <--

    n = len(series)
    if n < 2 * window:
        print(f"[AVISO] Série com {n} pontos < 2x janela interna ({window}). Retornando linha vazia.")
        return pd.DataFrame([{
            "block_height": block_height,
            "block_timestamp": block_timestamp,
            "shift_level_max": np.nan,
            "shift_var_max": np.nan,
            "shift_kl_max": np.nan
        }])
        # --> Série muito curta: retorna linha com NaNs / Too short: return row with NaNs <--

    shift_level = []
    shift_var = []
    shift_kl = []
    # --> Inicializa listas para mudanças / Initialize lists <--

    # ================================
    # LAÇOS DE JANELAS / LOOP THROUGH WINDOWS
    # ================================
    for i in range(n - 2 * window):
        win1 = series[i : i + window]
        win2 = series[i + window : i + 2 * window]
        # --> Divide a série em duas janelas consecutivas / Two consecutive windows <--

        shift_level.append(np.abs(np.mean(win1) - np.mean(win2)))
        shift_var.append(np.abs(np.var(win1) - np.var(win2)))

        hist1, _ = np.histogram(win1, bins=bins, density=True)
        hist2, _ = np.histogram(win2, bins=bins, density=True)
        shift_kl.append(kl_divergence(hist1, hist2))
        # --> Divergência KL entre distribuições normalizadas / KL divergence between histograms <--

    # ================================
    # CRIA DATAFRAME COM RESULTADO AGREGADO / CREATE AGGREGATED RESULT
    # ================================
    df_shifts = pd.DataFrame([{
        "block_height": block_height,
        "block_timestamp": block_timestamp,
        "shift_level_max": np.max(shift_level) if shift_level else np.nan,
        "shift_var_max": np.max(shift_var) if shift_var else np.nan,
        "shift_kl_max": np.max(shift_kl) if shift_kl else np.nan
    }])
    # --> Agrega máximos das métricas de mudança / Aggregate max values of shift metrics <--

    return df_shifts
    # --> Retorna DataFrame com uma linha de métricas / Returns one-row DataFrame with metrics <--

# ================================================================


In [ ]:
# EXECUÇÃO EM JANELAS DESLIZANTES / SLIDING WINDOW EXECUTION
# ================================================================
window_size = 60   # --> Tamanho da janela total / Total window size
step = 10          # --> Passo entre janelas / Step between windows
inner_window = 15  # --> Janela interna para comparação / Internal window size
shifts = []        # --> Lista para armazenar resultados / List to store results

for i in range(0, len(df_fft) - window_size + 1, step):
    sub_series = df_fft["btc_price_usd"].iloc[i:i + window_size]
    block_height = df_fft["block_height"].iloc[i + window_size - 1]
    block_timestamp = df_fft["block_timestamp"].iloc[i + window_size - 1]
    # --> Extrai informações finais da janela / Extract window metadata <--

    shift_feats = extract_structural_shifts(
        series=sub_series,
        block_height=block_height,
        block_timestamp=block_timestamp,
        window=inner_window  # --> Janela interna de comparação / Inner window size <--
    )

    shifts.append(shift_feats)

# ================================================================
# CONSOLIDAÇÃO FINAL / FINAL CONSOLIDATION
# ================================================================
if shifts:
    df_shifts = pd.concat(shifts, ignore_index=True)
    df_shifts = df_shifts.sort_values("block_timestamp")
    # --> Ordena resultado final / Sort final result <--
    display(df_shifts.head())
else:
    print("[AVISO] Nenhuma janela válida para extrair mudanças estruturais.")
    df_shifts = pd.DataFrame()
    # --> Se nenhuma janela for válida, retorna DataFrame vazio / If no valid window, return empty DataFrame <--

### 5. Análise de Estacionariedade e Transformações

#### 5.1 Transformações da Série

Diferenciação de Primeira e Segunda Ordem

A diferenciação é uma técnica para remover tendência e tornar a série mais estacionária.

- **Primeira diferença**:
$
Z'_t = Z_t - Z_{t-1}
$

- **Segunda diferença** (após aplicar a primeira):
$
Z''_t = Z'_t - Z'_{t-1} = Z_t - 2Z_{t-1} + Z_{t-2}
$

---

Comparação: Série Original vs Diferenciada

É importante visualizar e comparar:

- A série original  
- A série com primeira e segunda diferença  
- A estabilização da média e da variância

---

Transformação Box-Cox com \( \lambda \) ótimo via Guerrero

A transformação de Box-Cox torna a série **mais simétrica** e **estabiliza a variância**, essencial antes da diferenciação.

$
Y_t^{(\lambda)} = 
\begin{cases}
\frac{Y_t^\lambda - 1}{\lambda}, & \lambda \ne 0 \\
\ln(Y_t), & \lambda = 0
\end{cases}
$

O **método de Guerrero** encontra o valor de \( \lambda \) que minimiza a variância relativa das médias móveis de segmentos consecutivos.

---

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from scipy.stats import boxcox, boxcox_normmax
from scipy.special import boxcox1p
from sktime.transformations.series.boxcox import BoxCoxTransformer

# ================================================================
# TRANSFORMAÇÃO BOX-COX COM MLE / BOX-COX TRANSFORMATION VIA MLE
# ================================================================
def boxcox_mle(series: pd.Series) -> tuple:
    """
    Aplica Box-Cox com lambda ótimo via máxima verossimilhança (MLE)
    / Applies Box-Cox with optimal lambda via Maximum Likelihood Estimation
    """
    series = series.dropna()
    # --> Remove valores ausentes antes de aplicar a transformação / Drop missing values before transformation <--

    lambda_opt = boxcox_normmax(series, method="mle")
    # --> Estima o lambda ótimo via MLE / Estimate optimal lambda using MLE <--

    transformed = boxcox(series, lmbda=lambda_opt)
    # --> Aplica a transformação de Box-Cox / Apply Box-Cox transformation <--

    return transformed, lambda_opt
    # --> Retorna a série transformada e o lambda estimado / Return transformed series and lambda <--

# ================================================================
# TRANSFORMAÇÕES COMPLETAS DA SÉRIE / COMPLETE SERIES TRANSFORMATION
# ================================================================
def apply_series_transformations(series: pd.Series, seasonal_period: int = 12) -> dict:
    """
    Aplica transformações: Box-Cox, diferenças de 1ª e 2ª ordem
    / Applies Box-Cox, 1st and 2nd order differencing
    """
    series = series[series > 0].dropna()
    # --> Remove zeros ou valores negativos e NaNs (Box-Cox exige valores positivos) / Remove zeros, negatives, and NaNs <--

    boxcox_series, lambda_opt = boxcox_mle(series)
    # --> Aplica a transformação Box-Cox via MLE / Apply Box-Cox transformation using MLE <--

    diff1 = pd.Series(np.diff(boxcox_series), index=series.index[1:])
    # --> Calcula a 1ª diferença da série transformada / Compute 1st order differencing <--

    diff2 = pd.Series(np.diff(diff1), index=series.index[2:])
    # --> Calcula a 2ª diferença da série transformada / Compute 2nd order differencing <--

    return {
        "original": series,                                         # Série original / Original series
        "boxcox_transformed": pd.Series(boxcox_series, index=series.index),  # Série transformada / Transformed series
        "lambda_boxcox": lambda_opt,                                # Lambda ótimo estimado / Estimated optimal lambda
        "diff_1st": diff1,                                          # Primeira diferença / First difference
        "diff_2nd": diff2                                           # Segunda diferença / Second difference
    }

In [ ]:
result = apply_series_transformations(df_fft["btc_price_usd"])
print("Lambda ótimo:", result["lambda_boxcox"])


In [ ]:
result = apply_series_transformations(df_fft["btc_price_usd"])
plot_transformed_series(result)

#### 5.2 Diagnóstico de Estacionariedade

---

Análise Gráfica de ACF e PACF

A **Autocorrelação (ACF)** mostra o quanto a série atual depende de seus próprios lags.  
A **Autocorrelação Parcial (PACF)** mostra a correlação dos lags com a série, **removendo a influência dos lags anteriores**.

---

Teste Dickey-Fuller Aumentado (ADF)

Hipóteses:
- $ H_0 $: A série **possui raiz unitária** (não estacionária)
- $ H_1 $: A série **é estacionária**

Estatística de teste:  
$
\Delta Z_t = \alpha + \beta t + \gamma Z_{t-1} + \sum \delta_i \Delta Z_{t-i} + \varepsilon_t
$

---

Teste KPSS

Hipóteses:
- $ H_0 $: A série **é estacionária**
- $ H_1 $: A série **não é estacionária**

Complementa o ADF para evitar conclusões enviesadas.

---

Teste Phillips-Perron (PP)

Similar ao ADF, mas **mais robusto à heterocedasticidade e autocorrelação nos resíduos**.

---

Estimativas Ótimas de Diferença (unitroot_ndiffs)

- **unitroot_ndiffs** → número ideal de diferenciações para estacionarizar (não sazonal)
- **unitroot_nsdiffs** → número ideal de diferenciações sazonais

---

Rolling Média e Desvio

Verifica se a média e variância **permanecem estáveis ao longo do tempo**, outro indicativo visual de estacionariedade.

---

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss

# ================================================================
# ESTIMATIVA DE NDIFs PELO TESTE ADF / NDIFs ESTIMATION VIA ADF TEST
# ================================================================
def estimate_ndiffs_adf(series, max_d=3, alpha=0.05):
    """
    Estima o número mínimo de diferenciações necessárias para estacionarizar via ADF.
    / Estimates the minimum number of differences needed for stationarity using the ADF test.

    Parâmetros / Parameters:
    - series: pd.Series
        Série temporal a ser testada / Time series to be tested.
    - max_d: int
        Número máximo de diferenciações permitidas / Maximum number of differences allowed.
    - alpha: float
        Nível de significância para o teste ADF / Significance level for ADF test.

    Retorna / Returns:
    - int: número de diferenciações necessárias / Number of differences required.
    """
    for d in range(max_d + 1):
        test_series = series if d == 0 else series.diff(d).dropna()
        # --> Aplica diferenciação d vezes / Apply differencing d times <--

        pval = adfuller(test_series)[1]
        # --> Extrai o valor-p do teste ADF / Extract p-value from ADF test <--

        if pval < alpha:
            return d
        # --> Retorna o número de diferenças se a série for estacionária / Return d if stationary <--

    return max_d
    # --> Retorna o valor máximo se nenhuma diferenciação atender ao critério / Return max_d if no d meets criteria <--

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# ================================================================
# DIAGNÓSTICO DE ESTACIONARIEDADE / STATIONARITY DIAGNOSTICS
# ================================================================
def stationarity_diagnostics(
    series: pd.Series,
    block_height: int,
    block_timestamp: pd.Timestamp
) -> pd.DataFrame:
    """
    Executa diagnóstico básico de estacionariedade (ADF, KPSS e ndiffs).
    / Performs basic stationarity diagnostics (ADF, KPSS and estimated ndiffs).
    
    Parâmetros / Parameters:
    - series: pd.Series
        Série temporal a ser analisada / Time series to be analyzed.
    - block_height: int
        Altura do bloco associada à janela / Block height for traceability.
    - block_timestamp: pd.Timestamp
        Timestamp da última observação da janela / Timestamp of the last observation in the window.
    
    Retorna / Returns:
    - pd.DataFrame com uma linha e métricas de estacionariedade /
      One-row DataFrame with stationarity diagnostics.
    """

    # -----------------------
    # PRÉ-PROCESSAMENTO / PREPROCESSING
    # -----------------------
    series = series.dropna()
    # --> Remove valores ausentes da série / Drop missing values <--

    # -----------------------
    # TESTE ADF / ADF TEST
    # -----------------------
    adf_result = adfuller(series)
    adf_stat = adf_result[0]
    adf_pvalue = adf_result[1]

    # -----------------------
    # TESTE KPSS / KPSS TEST
    # -----------------------
    kpss_result = kpss(series, regression="c", nlags="legacy")
    kpss_stat = kpss_result[0]
    kpss_pvalue = kpss_result[1]

    # -----------------------
    # ESTIMATIVA DE NDIFs / ESTIMATED DIFFERENCES
    # -----------------------
    d_est = estimate_ndiffs_adf(series)

    # -----------------------
    # DATAFRAME DE RESULTADO / RESULT DATAFRAME
    # -----------------------
    return pd.DataFrame([{
        "block_height": block_height,
        "block_timestamp": block_timestamp,
        "ADF_stat": adf_stat,
        "ADF_pvalue": adf_pvalue,
        "KPSS_stat": kpss_stat,
        "KPSS_pvalue": kpss_pvalue,
        "ndiffs_est_adf": d_est
    }])

In [ ]:
from scipy.stats import boxcox

# Aplica Box-Cox
df_transf = df_fft.copy()
df_transf["boxcox_transformed"], _ = boxcox(df_transf["btc_price_usd"].dropna() + 1)

# Plota diagnósticos com a coluna criada
plot_rolling_diagnostics_overlay(
    series=df_transf["boxcox_transformed"],
    window=30,
    title="Rolling Média e Desvio - Box-Cox Transformada"
)

##### 🔶 ₿ -----> Diferenciação da Série

In [ ]:
# ============================
# CONFIGURAÇÕES DE PARÂMETROS
# ============================
resample_freq = 'h'            # Frequência: 'h' (hora), 'd' (dia), 'W' (semana) / Resampling frequency
nlags_option = 'auto'          # 'manual', 'serie_longa', 'auto' / Method to determine number of lags for ACF
manual_nlags = 30              # Lag manual, se escolhido / Manual lag count if selected
date_range_start = None        # Ex: "2023-01-01" / Optional start date filter
date_range_end = None          # Ex: "2023-03-01" / Optional end date filter

# ===========================
# PRÉ-PROCESSAMENTO DA SÉRIE
# ===========================

df_diff = df_features_temp.toPandas()
# --> Converte o DataFrame PySpark para Pandas / Converts PySpark DataFrame to Pandas <--

df_diff["block_timestamp"] = pd.to_datetime(df_diff["block_timestamp"])
# --> Garante que o campo de tempo seja do tipo datetime / Ensures timestamp field is datetime type <--

df_diff = df_diff.sort_values("block_timestamp")
# --> Ordena o DataFrame pelo tempo / Sorts the DataFrame by timestamp <--

df_diff.set_index('block_timestamp', inplace=True)
# --> Define o índice como timestamp (necessário para o resample) / Sets timestamp as index (needed for resample) <--

df_diff = df_diff.resample('d').mean()
# --> Reamostra os dados com frequência horária, calculando a média por hora / Resamples data to hourly frequency, taking the mean per hour <--


# ========================================
# FILTRO DE INTERVALO DE DATAS (OPCIONAL)
# ========================================
if date_range_start and date_range_end:
    df_diff = df_diff.loc[date_range_start:date_range_end]
# --> Filtra o DataFrame por intervalo de datas, se fornecido / Filters data by date range if defined <--

# =========================
# DIFERENCIAÇÃO (1ª ordem)
# =========================
df_diff["btc_price_diff"] = df_diff["btc_price_usd"].diff()
# --> Aplica diferenciação de 1ª ordem: Y(t) = Z(t) - Z(t-1) / First-order differencing <--

df_diff_stationary = df_diff.dropna()
# --> Remove os valores nulos gerados pela diferenciação / Drops NaNs from differencing <--

# ============================
# BUFFER VISUAL PARA GRÁFICOS
# ============================
y_min = df_diff["btc_price_usd"].min() * 0.98
y_max = df_diff["btc_price_usd"].max() * 1.02
# --> Define um intervalo visual com 2% de margem no eixo Y / Adds 2% buffer for Y-axis range <--

##### 🔶 ₿ -----> Modelagem Matemática

---
Condições de Estacionariedade de segunda ordem (Fraca)

Uma série temporal $ \ Z(t) \ $ é dita **fracamente estacionária** se satisfaz:

1. Média constante:  
   $
   \mathbb{E}[Z(t)] = \mu
   $

2. Variância constante:  
   $
   \text{Var}(Z(t)) = \sigma^2
   $

3. Autocovariância depende apenas do lag \( k \):  
   $
   \gamma(k) = \text{Cov}(Z(t), Z(t+k)) = \mathbb{E}[(Z(t) - \mu)(Z(t+k) - \mu)]
   $
---   

##### 🔶 ₿ -----> Plotagem da Série Original e Diferenciada

In [ ]:
plot_series_comparativa(
    df_graph=df_diff.reset_index(),  # <-- devolve 'block_timestamp' como coluna
    df_graph_stationary=df_diff_stationary.reset_index(),  # <-- idem
    y_min=y_min,
    y_max=y_max
)

#### 5.2 Diagnóstico de Estacionariedade

##### 🔶 ₿ -----> Modelagem Matemática

---
Função de Autocorrelação (ACF)


Dada uma série temporal $Z(t)$ com média $\mu$ e variância $\sigma^2$, a ACF no lag $k$ é definida como:

$
\rho(k) = \frac{\gamma(k)}{\gamma(0)} = \frac{\mathbb{E}[(Z(t) - \mu)(Z(t+k) - \mu)]}{\sigma^2}
$

Onde:  
$\rho(k)$ é o valor da ACF no lag $k$   
$\gamma(k)$ é a função de autocovariância no lag $k$  
$\gamma(0) = \text{Var}(Z(t)) = \sigma^2$  

---

##### 🔶 ₿ -----> Análise de Autocorrelação (ACF)

In [ ]:
df_acf = df_features_temp.toPandas()
# --> Converte o DataFrame do PySpark para Pandas / Converts PySpark DataFrame to Pandas <--

df_acf["block_timestamp"] = pd.to_datetime(df_acf["block_timestamp"])
# --> Garante que o campo de timestamp seja do tipo datetime / Ensures that 'block_timestamp' is in datetime format <--

df_acf = df_acf.sort_values("block_timestamp")
# --> Ordena os dados cronologicamente / Sorts the data by timestamp <--

df_acf.set_index('block_timestamp', inplace=True)
# --> Define 'block_timestamp' como índice do DataFrame (necessário para o resample) / Sets 'block_timestamp' as DataFrame index (required for resampling) <--

df_acf = df_acf.resample('h').mean()
# --> Reamostra os dados com frequência horária, calculando a média por hora / Resamples data to hourly frequency, taking the mean per hour <--

##### 🔶 ₿ -----> Diferenciação da Série

In [ ]:
# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

resample_freq = 'd'           # 'h' = hora, 'd' = dia, 'W' = semana / frequency of resampling
nlags_option = 'serie_longa'  # Opções: 'manual', 'auto', 'serie_longa' / lag selection mode
manual_nlags = 150            # Lag manual, se escolhido / manual lag value
date_range_start = None       # Ex: "2023-01-15" / optional start date
date_range_end = None         # Ex: "2023-02-15" / optional end date

# ===============
# PROCESSAMENTO
# ===============

df_acf = df_features_temp.toPandas()
# --> Converte o DataFrame PySpark para Pandas / Converts PySpark DataFrame to Pandas <--

df_acf["block_timestamp"] = pd.to_datetime(df_acf["block_timestamp"])
# --> Garante que o campo de tempo seja datetime / Ensures the timestamp is datetime <--

df_acf = df_acf.sort_values("block_timestamp").set_index("block_timestamp")
# --> Ordena os dados cronologicamente e define o índice / Sorts data chronologically and sets index <--

df_acf = df_acf.resample(resample_freq).mean()
# --> Reamostra a série conforme a frequência (diária neste caso) / Resamples the series (daily in this case) <--

if date_range_start and date_range_end:
    df_acf = df_acf.loc[date_range_start:date_range_end]
# --> Aplica filtro por intervalo de datas, se definido / Filters by date range, if defined <--

# Série diferenciada
serie_diff = df_acf["btc_price_usd"].diff().dropna()  # Y(t) = Z(t) - Z(t - 1)
# --> Aplica diferenciação de primeira ordem para obter série estacionária / Applies first-order differencing <--

# Seleção do número de defasagens
if nlags_option == 'manual':
    nlags = manual_nlags
    # --> Define lag manualmente / Uses manual lag value <--
elif nlags_option == 'serie_longa':
    nlags = len(serie_diff) // 4
    # --> Lag proporcional para séries longas / Uses one-fourth of the series length <--
elif nlags_option == 'auto':
    nlags = len(serie_diff)
    # --> Usa o comprimento total da série como nlags / Uses full length of the series <--
else:
    nlags = len(serie_diff)
    # --> Valor padrão / Default fallback value <--

##### 🔶 ₿ -----> Análise Gráfica da ACF

In [ ]:
plot_acf_diferenciada(serie_diff, nlags, nlags_option)

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import acf, pacf

# ================================================================
# EXTRAÇÃO DE ACF E PACF COM RASTREABILIDADE TEMPORAL
# ACF & PACF EXTRACTION WITH TRACEABILITY
# ================================================================
def extract_acf_pacf_features(
    series: pd.Series,
    block_height: pd.Series,
    block_timestamp: pd.Series,
    window: int = 24
) -> pd.DataFrame:
    """
    Extrai autocorrelações (ACF e PACF) em janelas móveis, com rastreabilidade temporal.
    / Extracts autocorrelation and partial autocorrelation in rolling windows with temporal traceability.

    Parâmetros / Parameters:
    - series: pd.Series
        Série temporal (ex: log-return) / Time series (e.g., log return).
    - block_height: pd.Series
        Altura do bloco para rastreamento / Block height for traceability.
    - block_timestamp: pd.Series
        Timestamp para rastreamento / Block timestamp for traceability.
    - window: int
        Tamanho da janela rolling / Rolling window size.

    Retorna / Returns:
    - pd.DataFrame com colunas: block_height, block_timestamp, acf_1, acf_5, acf_10, pacf_1
    """

    # -----------------------
    # Funções auxiliares para calcular ACF e PACF em cada janela
    # / Helper functions to compute ACF and PACF for a given window
    # -----------------------
    def calc_acf(x, lag):
        try:
            return acf(x, nlags=lag, fft=False)[lag]
        except Exception:
            return np.nan

    def calc_pacf(x, lag):
        try:
            return pacf(x, nlags=lag)[lag]
        except Exception:
            return np.nan

    # -----------------------
    # Inicializa DataFrame de resultados
    # / Initialize output DataFrame
    # -----------------------
    results = []

    for i in range(window, len(series)):
        window_series = series.iloc[i - window:i]
        bh = block_height.iloc[i]
        ts = block_timestamp.iloc[i]

        result = {
            "block_height": bh,
            "block_timestamp": ts,
            "acf_1": calc_acf(window_series, 1),
            "acf_5": calc_acf(window_series, 5),
            "acf_10": calc_acf(window_series, 10),
            "pacf_1": calc_pacf(window_series, 1)
        }
        results.append(result)

    # -----------------------
    # Converte lista em DataFrame final
    # / Convert result list into final DataFrame
    # -----------------------
    return pd.DataFrame(results)

In [ ]:
df_acf_pacf = extract_acf_pacf_features(
    series=df_returns["log_return"],
    block_height=df_returns["block_height"],
    block_timestamp=df_returns["block_timestamp"],
    window=48
)

display(df_acf_pacf.head())

##### 🔶 ₿ -----> Lembrete

---
Teste de Dickey-Fuller Aumentado (ADF)

O Teste ADF verifica se uma série temporal possui raiz unitária, ou seja, se não é estacionária.

A equação testada é:

$
\Delta Z_t = \alpha + \beta t + \gamma Z_{t-1} + \sum_{i=1}^{p} \delta_i \Delta Z_{t-i} + \varepsilon_t
$

Onde:
$Z_t$ → valor da série no tempo $t$  
$\Delta Z_t = Z_t - Z_{t-1}$ → primeira diferença  
$\alpha$ → constante (termo de tendência)  
$\beta t$ → tendência determinística (opcional)  
  
$\gamma$ → parâmetro-chave:  
Se $\gamma = 0$, a série tem raiz unitária (não estacionária)  
Se $\gamma < 0$, a série é estacionária  
$\sum \delta_i \Delta Z_{t-i}$ → componentes autorregressivos adicionais (lags)  
$\varepsilon_t$ → erro branco (white noise)  

⸻

Interpretação
	•	Hipótese nula ($H_0$): a série possui raiz unitária ⇒ não é estacionária
	•	Hipótese alternativa ($H_1$): a série é estacionária

Se p-valor < 0.05, rejeita-se $H_0$ ⇒ a série é estacionária.

---

##### 🔶 ₿ -----> Teste de Dickey-Fuller Aumentado (ADF)

In [ ]:
# ================================================================
# TESTE DE ESTACIONARIEDADE (ADF) COM SUPORTE A SÉRIES CURTAS
# ADF STATIONARITY TEST WITH SHORT SERIES SUPPORT
# ================================================================
def run_adf_test(series: pd.Series) -> dict:
    """
    Executa o teste de Dickey-Fuller Aumentado com proteção para séries curtas.
    / Runs Augmented Dickey-Fuller test with safe handling for short series.
    
    Retorna / Returns:
    - dicionário com estatística, p-valor e número de lags usados.
    / dictionary with test statistic, p-value, and number of lags used.
    """
    series = series.dropna()
    # --> Remove NaNs gerados pela diferenciação / Drop NaNs from differenced series <--

    if len(series) < 20:
        print(f"[AVISO] Série muito curta para ADF (n = {len(series)}). Retornando NaNs.")
        return {"adf_stat": np.nan, "p_value": np.nan, "used_lags": np.nan}
        # --> Evita execução do teste ADF com séries curtas / Prevents running ADF on short series <--

    try:
        resultado = adfuller(series)
        return {
            "adf_stat": resultado[0],
            "p_value": resultado[1],
            "used_lags": resultado[2]
        }
        # --> Retorna os resultados principais do teste ADF / Returns ADF main outputs <--
    except Exception as e:
        print(f"[ERRO] Falha no teste ADF: {e}")
        return {"adf_stat": np.nan, "p_value": np.nan, "used_lags": np.nan}
        # --> Em caso de erro inesperado / In case of error, return NaNs <--

# ================================================================
# PRÉ-PROCESSAMENTO DA SÉRIE PARA DIFERENCIAÇÃO
# PREPROCESSING SERIES FOR DIFFERENCING
# ================================================================
df_acf["btc_price_diff"] = df_acf["btc_price_usd"].diff()
# --> Aplica diferenciação de primeira ordem para tornar a série estacionária / Applies first-order differencing to stabilize the series <--

df_acf_stationary = df_acf.dropna()
# --> Remove os valores NaN resultantes da diferenciação / Drops NaN values from differencing <--

# ================================================================
# EXECUÇÃO DO TESTE ADF / RUNNING THE ADF TEST
# ================================================================
resultado_adf = run_adf_test(df_acf_stationary["btc_price_diff"])
# --> Executa o teste de estacionariedade com validação / Runs ADF test with safe fallback <--

# ================================================================
# RESULTADOS / RESULTS
# ================================================================
print("ADF:", resultado_adf["adf_stat"])
# --> Estatística do teste ADF (quanto mais negativa, mais estacionária a série) / ADF test statistic <--

print("p-valor:", resultado_adf["p_value"])
# --> p-valor do teste ADF (menor que 0.05 sugere estacionariedade) / ADF p-value <--

print("Usou até", resultado_adf["used_lags"], "lags")
# --> Número de defasagens consideradas no modelo ADF / Number of lags used in ADF model <--

##### 🔶 ₿ -----> Lembrete

---
Diagnóstico Visual de Estacionariedade — Média e Desvio Padrão Móveis

A série $Z(t)$ é transformada por diferenciação para análise de estacionariedade. Em seguida, calcula-se:
	1.	Média Móvel (Rolling Mean):  

$
\mu_t^{(w)} = \frac{1}{w} \sum_{i=0}^{w-1} Z(t - i)  
$

2.	Desvio Padrão Móvel (Rolling Std): 

$
\sigma_t^{(w)} = \sqrt{ \frac{1}{w} \sum_{i=0}^{w-1} \left(Z(t - i) - \mu_t^{(w)}\right)^2 }  
$

Onde:
$w$ = janela de tempo
$\mu_t^{(w)}$ = média móvel no tempo $t$  
$\sigma_t^{(w)}$ = desvio padrão móvel no tempo $t$  

⸻

Interpretação

Se ao longo do tempo:  
A média $\mu_t^{(w)}$ se mantém aproximadamente constante, e  
O desvio padrão $\sigma_t^{(w)}$ também permanece estável,  

Então a série diferenciada tende a ser fracamente estacionária (segunda ordem), o que é essencial para aplicar modelos como ARIMA, SARIMA ou GARCH.

---

##### 🔶 ₿ -----> Diagnóstico Visual de Estacionariedade — Média e Desvio Padrão Móveis

In [ ]:
# =============================
#  CONFIGURAÇÕES DE PARÂMETROS
# =============================
resample_freq = 'h'           # 'h' (hora), 'd' (dia), 'W' (semana)
window_size = 30              # Tamanho da janela de média/desvio / Rolling window size
date_range_start = None       # Ex: "2023-01-01"
date_range_end = None         # Ex: "2023-03-01"

# ===============
#  PROCESSAMENTO
# ===============

df_Rolling = df_features_temp.toPandas()
# --> Converte o DataFrame PySpark para Pandas / Converts PySpark DataFrame to Pandas <--

df_Rolling["block_timestamp"] = pd.to_datetime(df_Rolling["block_timestamp"])
# --> Garante que o campo de tempo seja do tipo datetime / Ensures timestamp is datetime type <--

df_Rolling = df_Rolling.sort_values("block_timestamp")
# --> Ordena os dados por tempo crescente / Sorts data by time ascending <--

df_Rolling = df_Rolling.set_index("block_timestamp").resample(resample_freq).mean()
# --> Reamostra os dados com base na frequência definida (ex: 'h' para hora) / Resamples the data based on the defined frequency <--

if date_range_start and date_range_end:
    df_Rolling = df_Rolling.loc[date_range_start:date_range_end]
# --> Filtra a série pelo intervalo de datas, se definido / Filters series by date range if provided <--

df_Rolling["block_timestamp"] = df_Rolling.index
# --> Restaura a coluna de timestamp para uso em gráficos / Restores timestamp column for plotting <--

df_Rolling["btc_price_diff"] = df_Rolling["btc_price_usd"].diff()
# --> Aplica diferenciação de primeira ordem / Applies first-order differencing <--

df_Rolling = df_Rolling.dropna()
# --> Remove os valores nulos gerados pela diferenciação / Drops NaN values caused by differencing <--

# =================================
#  CÁLCULO DE MÉDIA E DESVIO MÓVEL
# =================================

rolling_mean = df_Rolling["btc_price_diff"].rolling(window=window_size).mean()
# --> Calcula a média móvel com janela definida / Computes rolling mean with defined window <--

rolling_std = df_Rolling["btc_price_diff"].rolling(window=window_size).std()
# --> Calcula o desvio padrão móvel com janela definida / Computes rolling standard deviation <--

In [ ]:
plot_rolling_mean_std(df_Rolling, rolling_mean, rolling_std)

##### 🔶 ₿ -----> Lembrete

---
Teste KPSS — Validação de Estacionariedade  

O teste KPSS (Kwiatkowski–Phillips–Schmidt–Shin) avalia a hipótese nula de que a série é estacionária (em nível ou tendência).  

A série temporal $Z_t$ é modelada como:  

$
Z_t = r_t + u_t  
$

Onde:
$r_t$ é uma tendência determinística (constante ou linear)
$u_t$ é um processo estacionário

⸻

Estatística do teste:

$
\text{KPSS} = \frac{1}{T^2} \sum_{t=1}^{T} S_t^2 \bigg/ \hat{\sigma}^2  
$

Com:
$S_t = \sum_{i=1}^{t} \hat{u}_i$: soma acumulada dos resíduos  
$\hat{\sigma}^2$: estimativa da variância dos resíduos via Newey-West  
$T$: número total de observações  

---

##### 🔶 ₿ -----> Teste KPSS — Teste de Raiz Unitária

In [ ]:
# =================================
# TESTE DE ESTACIONARIEDADE (KPSS)
# =================================

# Suprimir apenas o InterpolationWarning
from statsmodels.tools.sm_exceptions import InterpolationWarning

with warnings.catch_warnings():
    warnings.simplefilter("ignore", InterpolationWarning)
    # --> Suprime o aviso de interpolação gerado pelo KPSS / Suppresses interpolation warning from KPSS <--

    stat, p_value, lags, crit = kpss(df_acf["btc_price_diff"].dropna(), regression='c')
    # --> Executa o teste KPSS na série diferenciada com tendência constante ('c') / Runs KPSS test with constant trend assumption <--

print("KPSS Estatística:", stat)
# --> Estatística do teste KPSS (valores maiores indicam evidência contra estacionariedade) 
# --> KPSS test statistic (higher values indicate non-stationarity) <--

print("p-valor:", p_value)
# --> p-valor do teste KPSS (p < 0.05 indica rejeição da estacionariedade) / KPSS p-value (p < 0.05 suggests non-stationarity) <--

##### 🔶 ₿ -----> Lembrete

---
Função de Autocorrelação Parcial (PACF)  

A Função de Autocorrelação Parcial (PACF) mede a correlação entre os valores da série temporal com uma defasagem k, eliminando a influência dos lags intermediários.  

Ou seja, a PACF no lag k representa a correlação entre Z(t) e Z(t-k) condicionada aos valores entre eles Z(t-1), Z(t-2), …, Z(t-k+1).  

Para uma série temporal Z(t), a PACF no lag k é definida como o coeficiente \phi_{kk} na regressão linear:  

$
Z(t) = \phi_{k1}Z(t-1) + \phi_{k2}Z(t-2) + \cdots + \phi_{kk}Z(t-k) + \varepsilon_t  
$

Onde:  
$\phi_{kk}$ é a autocorrelação parcial no lag k   
$\varepsilon_t$ é o erro branco (resíduo aleatório)  

A PACF ajuda a identificar a ordem p de um modelo AR(p), pois mostra claramente até onde as correlações são significativas sem efeito indireto de lags intermediários.  

---

##### 🔶 ₿ -----> Verificação de Autocorrelação/Padrões com ACF x PACF

In [ ]:
plot_acf_pacf(df_features_temp, resample_freq='h', nlags_option='serie_longa', manual_nlags=150, 
                  date_range_start=None, date_range_end=None)

In [ ]:
# ================================================================
# PARÂMETROS GERAIS / GENERAL PARAMETERS
# ================================================================
window_size = 60     # --> Tamanho da janela deslizante / Sliding window size <--
step = 10            # --> Passo da janela / Window step <--
stationarity_results = []  # --> Lista para armazenar resultados / List to store results <--

# ================================================================
# LOOP DE EXTRAÇÃO / FEATURE EXTRACTION LOOP
# ================================================================
for i in range(0, len(df_fft) - window_size + 1, step):
    # Subconjunto da série para análise local
    # / Local series slice for feature extraction
    janela = df_fft["btc_price_usd"].iloc[i:i + window_size]

    # Dados de rastreio associados à última posição da janela
    # / Metadata from the end of the window
    block_height = df_fft["block_height"].iloc[i + window_size - 1]
    block_timestamp = df_fft["block_timestamp"].iloc[i + window_size - 1]

    # ================================
    # SUPRESSÃO DE WARNINGS / WARNING SUPPRESSION
    # ================================
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=InterpolationWarning)
        warnings.simplefilter("ignore", category=UserWarning)

        try:
            # ================================
            # CÁLCULO DAS FEATURES DE ESTACIONARIEDADE
            # STATIONARITY FEATURE EXTRACTION
            # ================================
            result = stationarity_diagnostics(
                series=janela,
                block_height=block_height,
                block_timestamp=block_timestamp
            )
            # --> Executa função de extração robusta para séries curtas e longas / Calls robust function with short/long support <--

        except Exception as e:
            print(f"[ERRO] Falha ao extrair métricas na janela {i}:{i+window_size} — {e}")
            # --> Em caso de falha, cria linha com NaNs / On failure, fallback to NaN <--
            result = pd.DataFrame([{
                "block_height": block_height,
                "block_timestamp": block_timestamp,
                "adf_stat": np.nan,
                "adf_pvalue": np.nan,
                "kpss_stat": np.nan,
                "kpss_pvalue": np.nan
            }])

    stationarity_results.append(result)

# ================================================================
# CONSOLIDAÇÃO FINAL / FINAL CONSOLIDATION
# ================================================================
if stationarity_results:
    stationarity = pd.concat(stationarity_results, ignore_index=True)
    # --> Concatena todos os DataFrames extraídos / Concatenates all result DataFrames <--
    display(stationarity.head())
else:
    print("[AVISO] Nenhuma janela válida para calcular estacionariedade.")
    stationarity = pd.DataFrame()
    # --> Retorna DataFrame vazio se nada foi extraído / Returns empty DataFrame if no extraction succeeded <--

### 6. Pré-Modelagem e Ajustes Estatísticos

#### 6.1 Modelos Lineares de Benchmark

##### 🔶 ₿ -----> Grid Search - ARIMA

In [ ]:
# Suprime warnings de modelos que não convergem
warnings.filterwarnings("ignore")
# --> Suprime avisos durante o ajuste de modelos, especialmente de convergência / Suppresses warnings (e.g., convergence) during model fitting <--

# ==============
# CONFIGURAÇÕES
# ==============

p_range = range(0, 4)  # Ex: 0 a 3
d = 1                  # Grau de diferenciação fixo
q_range = range(0, 4)

melhor_aic = float("inf")
# --> Inicializa o melhor AIC com infinito para garantir substituição / Initializes best AIC as infinity for comparison <--

melhor_ordem = None
melhor_modelo = None
# --> Armazena a melhor ordem (p,d,q) e o melhor modelo ajustado / Stores best order and model instance <--

# ====================
# GRID SEARCH (p,d,q)
# ====================

for p in p_range:
    for q in q_range:
        try:
            modelo = ARIMA(serie_diff, order=(p, d, q)).fit()
            # --> Ajusta o modelo ARIMA para combinação atual de (p,d,q) / Fits ARIMA model for current (p,d,q) combination <--

            aic = modelo.aic
            # --> Extrai o valor de AIC do modelo ajustado / Retrieves AIC from fitted model <--

            if aic < melhor_aic:
                melhor_aic = aic
                melhor_ordem = (p, d, q)
                melhor_modelo = modelo
            # --> Atualiza se o AIC for o menor encontrado até agora / Updates best model if AIC is lower <--

        except Exception:
            continue
        # --> Ignora erros em modelos que não convergem / Skips models that raise errors <--

# ==========
# RESULTADO
# ==========

print(f"Melhor ordem (p,d,q): {melhor_ordem}")
# --> Exibe a melhor combinação de parâmetros encontrada / Prints best order (p,d,q) found <--

print(f"Menor AIC: {melhor_aic:.2f}")
# --> Exibe o menor valor de AIC alcançado / Prints lowest AIC achieved <--

##### 🔶 ₿ -----> Modelagem Matemática

---
Modelagem Matemática — ARIMA (p, d, q)

O modelo ARIMA (AutoRegressive Integrated Moving Average) é uma combinação de três componentes:  
p: número de termos autorregressivos (AR)  
d: número de diferenciações necessárias para tornar a série estacionária  
q: número de termos de média móvel (MA)  

⸻

A fórmula geral do modelo ARIMA(p, d, q) é:  

$
\phi(B)(1 - B)^d Z_t = \theta(B) \varepsilon_t
$

Onde:  
$Z_t$: valor da série temporal no tempo t  
$B$: operador defasagem (lag), ou seja BZ_t = Z_{t-1}  
$(1 - B)^d$: parte de diferenciação (para tornar estacionária)  
$\varepsilon_t$: termo de erro (ruído branco)  
$\phi(B) = 1 - \phi_1 B - \phi_2 B^2 - \dots - \phi_p B^p$: parte AR  
$\theta(B) = 1 + \theta_1 B + \theta_2 B^2 + \dots + \theta_q B^q$: parte MA  

⸻

Interpretação:  
Se $d = 1$, estamos modelando a primeira diferença da série $ Y_t = Z_t - Z_{t-1}$.  
O ARIMA então busca capturar padrões no comportamento das variações, não dos valores absolutos.  
Um bom modelo resulta em resíduos não correlacionados, ou seja, ruído branco.  

---

##### 🔶 ₿ -----> ARIMA

In [ ]:
p, d, q = melhor_ordem  # Ex: (1, 1, 1)
# --> Atribui os melhores parâmetros encontrados pelo Grid Search / Assigns best (p,d,q) from Grid Search <--

# Ajustar modelo ARIMA com a série diferenciada
modelo = ARIMA(serie_diff, order=(p, d, q))
# --> Inicializa o modelo ARIMA com a série diferenciada e a ordem selecionada / Initializes ARIMA model with differenced series and selected order <--

modelo_ajustado = modelo.fit()
# --> Ajusta o modelo aos dados / Fits the model to the data <--

print(modelo_ajustado.summary())
# --> Mostra o resumo estatístico completo do modelo ARIMA ajustado / Displays full statistical summary of the fitted ARIMA model <--

##### 🔶 ₿ -----> Extração de Camadas Extraídas - ARIMA

In [ ]:
plot_arima_layers(df_acf, p, d, q)

In [ ]:
# Seleciona uma janela da série de preços
janela = df_fft["btc_price_usd"].iloc[-60:]

# Define as chaves de rastreio (último ponto da janela)
block_height = df_fft["block_height"].iloc[-1]
block_timestamp = df_fft["block_timestamp"].iloc[-1]

# Executa a extração ARIMA com rastreabilidade
df_arima_features = extract_arima_features(
    series=janela,
    block_height=block_height,
    block_timestamp=block_timestamp,
    order=(1, 1, 1)
)

# Visualiza o resultado
display(df_arima_features.tail())

In [ ]:
df_regime = extract_regime_features(df_fft, col="btc_price_usd", window=48)
display(df_regime.tail())

#### 6.2 Diagnosticos de Residuos

##### 🔶 ₿ -----> ACF dos Resíduos

In [ ]:
plot_acf_residuos(modelo_ajustado, nlags=40)

##### 🔶 ₿ -----> Teste de LJUNG-BOX

In [ ]:
# ================================================================
# TESTE DE LJUNG-BOX PARA RESÍDUOS / LJUNG-BOX TEST FOR RESIDUALS
# ================================================================
def run_ljung_box_test(residuals, lags=10):
    """
    Executa o teste de Ljung-Box com tratamento para séries curtas.
    / Runs Ljung-Box test with fallback for short residual series.

    Parâmetros / Parameters:
    - residuals: pd.Series ou np.array
        Série de resíduos do modelo ajustado / Residual series from fitted model
    - lags: int
        Número de defasagens para o teste / Number of lags in the test

    Retorna / Returns:
    - dict com estatística e p-valor do teste
    / dict with test statistic and p-value
    """
    residuals = pd.Series(residuals).dropna()
    # --> Remove valores ausentes dos resíduos / Drop NaNs from residuals <--

    if len(residuals) <= lags:
        print(f"[AVISO] Série de resíduos muito curta (n = {len(residuals)}) para {lags} lags. Retornando NaNs.")
        return {"lb_stat": np.nan, "p_value": np.nan}
        # --> Série insuficiente para o teste / Not enough residuals for the test <--

    try:
        ljung_result = acorr_ljungbox(residuals, lags=[lags], return_df=True)
        return {
            "lb_stat": ljung_result["lb_stat"].iloc[0],
            "p_value": ljung_result["lb_pvalue"].iloc[0]
        }
        # --> Executa e retorna os resultados / Run test and return result <--
    except Exception as e:
        print(f"[ERRO] Falha no teste de Ljung-Box: {e}")
        return {"lb_stat": np.nan, "p_value": np.nan}
        # --> Em caso de falha, retorna NaNs / On failure, return NaNs <--

# ================================================================
# EXEMPLO DE APLICAÇÃO / APPLICATION EXAMPLE
# ================================================================
residuos = modelo_ajustado.resid.dropna()
# --> Obtém os resíduos do modelo e remove valores nulos / Extracts residuals and drops NaN values <--

ljung_result = run_ljung_box_test(residuos, lags=10)
# --> Executa o teste de Ljung-Box com fallback para séries curtas / Runs Ljung-Box with fallback <--

print("Resultado do Teste de Ljung-Box:")
print(f"Estatística: {ljung_result['lb_stat']}")
print(f"p-valor: {ljung_result['p_value']}")
# --> Exibe estatísticas e p-valor para autocorrelação conjunta / Shows p-value for joint autocorrelation <--

# ================================
# INTERPRETAÇÃO / INTERPRETATION
# ================================
# Se p-valor > 0.05 → resíduos não têm autocorrelação → modelo é adequado.
# Se p-valor < 0.05 → evidência de autocorrelação nos resíduos → modelo pode ser melhorado.

##### 🔶 ₿ -----> Histograma dos resíduos + Curva Normal

In [ ]:
plot_residuos_analysis(residuos)

### 7. Consolidação das Features Temporais